In [ ]:
rm -r progettoML

rm: cannot remove 'progettoML': No such file or directory


In [1]:
#!wget https://github.com/git-lfs/git-lfs/releases/download/v2.9.0/git-lfs-linux-amd64-v2.9.0.tar.gz
#!tar -xf git-lfs-linux-amd64-v2.9.0.tar.gz
#!chmod 755 install.sh
#!./install.sh
#!git clone -b wikitrain https://github.com/gioanat/progettoML
!git clone https://github.com/gioanat/progettoML

Cloning into 'progettoML'...
remote: Enumerating objects: 522, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 522 (delta 57), reused 0 (delta 0), pack-reused 397
Receiving objects: 100% (522/522), 447.57 MiB | 27.42 MiB/s, done.
Resolving deltas: 100% (168/168), done.
Checking out files: 100% (317/317), done.


In [ ]:
import os
import subprocess
if not os.path.isdir("data/wikitext-103/"):
    print("Downloading data...")
    subprocess.run("wget -c https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip -P data".split())
    print("Unzipping data...")
    subprocess.run(["unzip", "data/wikitext-103-v1.zip", "-d", "data/"])
    print("Done...")
else:
    print("Found data...")

Unzipping data...
Done...


In [ ]:
class TableHelper(object):
    def __init__(self, row_count: int, col_count: int):
        self.grid = [[None for _ in range(col_count)]
                     for _ in range(row_count)]

    #  column_name permette di indicizzare la colonna a partire dal valore della cella [0]
    # se è definito, ha priorità più alta rispetto a column_index
    def set_column(self, column_index: int,  items: list, col_name=None):
        if col_name != None:
            temp = -1
            for i in range(len(self.grid[0])):
                if self.grid[0][i] == col_name:
                    temp = i
                    break
            if temp == -1:
                print("Non esiste nessuna colonna chiamata:", col_name)
            column_index = temp
        for row_index in range(min(len(self.grid), len(items))):
            self.grid[row_index][column_index] = items[row_index]

    def set_row(self, row_index: int, items: list, row_name=None):
        if row_name != None:
            temp = -1
            for i in range(len(self.grid)):
                if self.grid[i][0] == row_name:
                    temp = i
                    break
            if temp == -1:
                print("Non esiste nessuna riga chiamata:", row_name)
            row_index = temp
            
        if len(items) != len(self.grid[0]):
            raise Exception("Non posso aggiungere un numero di elementi diverso dalla grandezza della tabella: {} != {}".format(
                len(items), len(self.grid[0])))

        self.grid[row_index] = items

    def set_cell(self,  item: object, row_name=None, col_name=None, row_index=None, col_index=None):
        if col_name == None and col_index == None:
            raise Exception(
                "Almeno uno fra col_name e col_index dev'essere valorizzato")

        if row_name == None and row_index == None:
            raise Exception(
                "Almeno uno fra row_name e row_index dev'essere valorizzato")

        if row_name != None:
            temp = -1
            for i in range(len(self.grid)):
                if self.grid[i][0] == row_name:
                    temp = i
                    break
            if temp == -1:
                print("Non esiste nessuna riga chiamata:", row_name)
            row_index = temp

        if col_name != None:
            temp = -1
            for i in range(len(self.grid[0])):
                if self.grid[0][i] == col_name:
                    temp = i
                    break
            if temp == -1:
                print("Non esiste nessuna colonna chiamata:", col_name)
            col_index = temp

        if col_index == None and row_index == None:
            raise Exception("Non è possibile aggiungere la cella ({},{}) o ({},{})".format(
                row_index, col_index, row_name, col_name))

        if 0 <= row_index < len(self.grid):
            if 0 <= col_index < len(self.grid[0]):
                self.grid[row_index][col_index] = item
                return

        raise Exception(
            "La cella ({}, {}) non è presente nella tabella".format(row_index, col_index))

    def print(self):
        max_length = 0
        for row_index in range(len(self.grid)):
            for col_index in range(len(self.grid[row_index])):
                cell = self.grid[row_index][col_index]
                if cell is None:
                    max_length = max(max_length, 4)
                    self.grid[row_index][col_index] = ''
                else:
                    max_length = max(max_length, len(str(cell)))

        size = max_length * len(self.grid[0]) + (len(self.grid[0]) + 1)

        print('-'*size)
        for row in self.grid:
            format = ""
            [print(format+f'|{{:^{max_length}}}'.format(item), end="")
             for item in row]
            print("|")
        print('-'*size)


table = TableHelper(row_count=5, col_count=5)
table.set_column(column_index=0, items=[
                 None, 'BERT base', 'GPT', 'TFXL', 'WT103'])
table.set_cell(row_index=0, col_index=1, item='Corpus-BLEU WT103')
table.set_cell(row_index=0, col_index=2, item='PPL')
table.set_cell(row_index=0, col_index=3, item='Corpus-PPL')
table.set_cell(row_index=0, col_index=4, item='EmbSim WT103')


#table.print()

gramTable = TableHelper(row_count=5, col_count=8)
gramTable.set_column(column_index=0, items=[
    None, 'BERT base','GPT', 'TFXL', 'WT103'])
gramTable.set_cell(row_index=0, col_index=1, item='Self-BLEU')
gramTable.set_cell(row_index=0, col_index=2, item='Self unique 2-grams')
gramTable.set_cell(row_index=0, col_index=3, item='Self unique 3-grams')
gramTable.set_cell(row_index=0, col_index=4, item='Self unique 4-grams')
gramTable.set_cell(row_index=0, col_index=5, item='WT103 unique 2-grams')
gramTable.set_cell(row_index=0, col_index=6, item='WT103 unique 3-grams')
gramTable.set_cell(row_index=0, col_index=7, item='WT103 unique 4-grams')

#gramTable.set_cell(item="prova", row_name="GPT",col_name="Self unique 3-grams")

#gramTable.print()

In [ ]:
tableMLE = TableHelper(row_count=3, col_count=5)
tableMLE.set_column(column_index=0, items=[
                 None, 'MLE', 'WT103MLE'])
tableMLE.set_cell(row_index=0, col_index=1, item='Corpus-BLEU WT103')
tableMLE.set_cell(row_index=0, col_index=2, item='PPL')
tableMLE.set_cell(row_index=0, col_index=3, item='Corpus-PPL')
tableMLE.set_cell(row_index=0, col_index=4, item='EmbSim')


#table.print()

gramTableMLE = TableHelper(row_count=3, col_count=8)
gramTableMLE.set_column(column_index=0, items=[
    None, 'MLE', 'WT103MLE'])
gramTableMLE.set_cell(row_index=0, col_index=1, item='Self-BLEU')
gramTableMLE.set_cell(row_index=0, col_index=2, item='Self unique 2-grams')
gramTableMLE.set_cell(row_index=0, col_index=3, item='Self unique 3-grams')
gramTableMLE.set_cell(row_index=0, col_index=4, item='Self unique 4-grams')
gramTableMLE.set_cell(row_index=0, col_index=5, item='WT103 unique 2-grams')
gramTableMLE.set_cell(row_index=0, col_index=6, item='WT103 unique 3-grams')
gramTableMLE.set_cell(row_index=0, col_index=7, item='WT103 unique 4-grams')

# BERT Has a Mounth and it must speaks:

---

Importiamo le librerie:

In [ ]:
try:
  from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
except:
  import sys
  !{sys.executable} -m pip install pytorch_pretrained_bert
  from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

     |████████████████████████████████| 133kB 8.2MB/s 
     |████████████████████████████████| 133kB 13.4MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 7.6MB 11.3MB/s 
ERROR: botocore 1.20.97 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


## BERT-base

Carichiamo i pesi del modello 'bert-base-uncased' nel caso di lingua inglese, e 'bert-base-italian-cased' nel caso di lingua italiana. Furthermore, we take tracking of index of special symbols, like [MASK], [CLS] (start of sentences) and [SEP] (separator of two not consecutive sentences).

In [ ]:
italian = False
# Load pre-trained model (weights)
model_version = 'bert-base-uncased'
model = BertForMaskedLM.from_pretrained(model_version)
model.eval()
cuda = torch.cuda.is_available()
if cuda:
    model = model.cuda(0)
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=model_version.endswith("uncased"))

CLS = '[CLS]'
SEP = '[SEP]'
MASK = '[MASK]'
mask_id = tokenizer.convert_tokens_to_ids([MASK])[0]
sep_id = tokenizer.convert_tokens_to_ids([SEP])[0]
cls_id = tokenizer.convert_tokens_to_ids([CLS])[0]

100%|██████████| 231508/231508 [00:00<00:00, 1195775.24B/s]


In this chunk, we define some functions that allow us to obtain the index of the given token in the vocabulary and vice versa and to detokenize generated sentences. 

In [ ]:
def tokenize_batch(batch):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def untokenize_batch(batch):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

This function generates a word and locates it in gen_idx position. Given tensor of logits of the batch, this function generates a new word:



*   sampling from the top $k$ most probable words if 'top_k' parameter is different from zero;
*   sampling from full distribution if 'sample' is equals to 'True';
*   otherwise, the most probable words is chosen.

Furthemore, if temperature parameter is given in input, then it is uses with combination of softmax function, for retrieving the probabilities.

\begin{equation}
\text{softmax}(z)_i = \frac{e^{\frac{z_i}{T}}}{\sum_{j=1}^N e^{\frac{z_j}{T}}}
\end{equation}


So,temperature is a hyperparameter which is applied to logits to affect the final probabilities from the softmax. In particular

*   A low temperature (below 1) makes the model more confident.
*   A high temperature (above 1) makes the model less confident.

In [ ]:
def generate_step(out, gen_idx, temperature=None, top_k=0, sample=False, return_list=True):
    """ Generate a word from from out[gen_idx]
    
    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k 
    """
    if italian:
      logits = out.logits[:,gen_idx]
    else:
      logits = out[:,gen_idx]
    if temperature is not None:
        logits = logits / temperature
    if top_k > 0:
        kth_vals, kth_idx = logits.topk(top_k, dim=-1)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        idx = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1)
    elif sample:
        dist = torch.distributions.categorical.Categorical(logits=logits)
        idx = dist.sample().squeeze(-1)
    else:
        idx = torch.argmax(logits, dim=-1)
    return idx.tolist() if return_list else idx

This is the meat of the algorithm. The general idea is

1.   start from all masks (get_init_text);
2.   repeatedly pick a location, mask the token at that location, and generate from the probability distribution given by BERT;
3.   stop when converged or tired of waiting

We consider three "modes" of generating:

*    generate a single token for a position chosen uniformly at random for a chosen number of time steps (parallel_sequential_generation);
*    generate in sequential order (L->R), one token at a time (sequential_generation);
*    generate for all positions at once for a chosen number of time steps (parallel_generation).

The generate function wraps and batches these three generation modes. In practice, we find that the first leads to the most fluent samples.

In [ ]:
# Generation modes as functions
import math
import time

def get_init_text(seed_text, max_len, batch_size = 1, rand_init=False):
    """ Get initial sentence by padding seed_text with either masks or random words to max_len """
    batch = [seed_text + [MASK] * max_len + [SEP] for _ in range(batch_size)]  # crea batch_size sentences of max_len that start with [CLS] and end with [SEP]
    #if rand_init:
    #    for ii in range(max_len):
    #        init_idx[seed_len+ii] = np.random.randint(0, len(tokenizer.vocab))
    
    return tokenize_batch(batch)

def parallel_sequential_generation(seed_text, batch_size=10, max_len=15, top_k=0, temperature=None, max_iter=300, burnin=200,
                                   cuda=False, print_every=10, verbose=True):
    """ Generate for one random position at a timestep
    
    args:
        - burnin: during burn-in period, sample from full distribution; afterwards take argmax
    """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_iter):
        kk = np.random.randint(0, max_len) # seleziona un indice casuale tra 0 e max_len e maschera il relativo token in ogni sentences
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = mask_id
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        topk = top_k if (ii >= burnin) else 0
        idxs = generate_step(out, gen_idx=seed_len+kk, top_k=topk, temperature=temperature, sample=(ii < burnin))
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = idxs[jj]
            
        if verbose and np.mod(ii+1, print_every) == 0:
            for_print = tokenizer.convert_ids_to_tokens(batch[0])
            for_print = for_print[:seed_len+kk+1] + ['(*)'] + for_print[seed_len+kk+1:]
            print("iter", ii+1, " ".join(for_print))
            
    return untokenize_batch(batch)

def parallel_generation(seed_text, batch_size=10, max_len=15, top_k=0, temperature=None, max_iter=300, sample=True, 
                        cuda=False, print_every=10, verbose=True):
    """ Generate for all positions at each time step """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_iter):
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        for kk in range(max_len):
            idxs = generate_step(out, gen_idx=seed_len+kk, top_k=top_k, temperature=temperature, sample=sample)
            for jj in range(batch_size):
                batch[jj][seed_len+kk] = idxs[jj]
            
        #if verbose and np.mod(ii, print_every) == 0:
        #    print("iter", ii+1, " ".join(tokenizer.convert_ids_to_tokens(batch[0])))
    
    return untokenize_batch(batch)
            
def sequential_generation(seed_text, batch_size=10, max_len=15, leed_out_len=15, 
                          top_k=0, temperature=None, sample=True, cuda=False):
    """ Generate one word at a time, in L->R order """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size) # batch_size sentences of max_len length with all mask, starting with [CLS] and separating by [SEP]
    
    for ii in range(max_len):
        inp = [sent[:seed_len+ii+leed_out_len]+[sep_id] for sent in batch]
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp) # ricava i valori logits
        idxs = generate_step(out, gen_idx=seed_len+ii, top_k=top_k, temperature=temperature, sample=sample)
        for jj in range(batch_size):
            batch[jj][seed_len+ii] = idxs[jj]
        
    return untokenize_batch(batch)


def generate(n_samples, seed_text="[CLS]", batch_size=10, max_len=25, 
             generation_mode="parallel-sequential",
             sample=True, top_k=100, temperature=1.0, burnin=200, max_iter=500,
             cuda=False, print_every=1):
    # main generation function to call
    sentences = []
    n_batches = math.ceil(n_samples / batch_size) # approssima il rapporto a un numero intero e ripete il processo di generazione di batch_size sentences of length len_max n_batches times.
    start_time = time.time()
    for batch_n in range(n_batches):
        if generation_mode == "parallel-sequential":
            batch = parallel_sequential_generation(seed_text, batch_size=batch_size, max_len=max_len, top_k=top_k,
                                                   temperature=temperature, burnin=burnin, max_iter=max_iter, 
                                                   cuda=cuda, verbose=False)
        elif generation_mode == "sequential":
            batch = sequential_generation(seed_text, batch_size=batch_size, max_len=max_len, top_k=top_k, 
                                          temperature=temperature, leed_out_len=leed_out_len, sample=sample,
                                          cuda=cuda)
        elif generation_mode == "parallel":
            batch = parallel_generation(seed_text, batch_size=batch_size,
                                        max_len=max_len, top_k=top_k, temperature=temperature, 
                                        sample=sample, max_iter=max_iter, 
                                        cuda=cuda, verbose=True)
        
        if (batch_n + 1) % print_every == 0:
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time))
            start_time = time.time()
        
        sentences += batch
    return sentences

In [ ]:
# Utility functions

def printer(sent, should_detokenize=True,otherModes =False):
    if otherModes:
        sent = detokenize(sent[1:-1])
    if should_detokenize:
        sent = detokenize(sent)
    print(" ".join(sent))
    
def read_sents(in_file, should_detokenize=False,openaiFlag = True):
    if openaiFlag:
      sents = [sent.replace("<EOS> ","").strip().split() for sent in open(in_file).readlines()]
      if should_detokenize:
          sents = [detokenize(sent) for sent in sents]
      return sents
    else:
      sents = [sent.replace("<unk>","[UNK]").strip().split() for sent in open(in_file).readlines()]
      if should_detokenize:
          sents = [detokenize(sent) for sent in sents]
      return sents

def write_sents(out_file, sents, should_detokenize=False,bert=True):
    with open(out_file, "w") as out_fh:
        for sent in sents:
          if bert:
            sent = detokenize(sent[1:-1]) if should_detokenize else sent
          else:  
            sent = detokenize(sent) if should_detokenize else sent
          out_fh.write("%s\n" % " ".join(sent))

Let's call the actual generation function! We must set the following:

- batch_size: number of generated sentences;
- max_len: max length of sequence to generate (equals to number of generated tokens)
- n_samples: it generates n_samples/batch_size (rounded) block of batch_size sentences of length max_len (only for 'sequential' mode);
- top_k: at each step, sample from the top_k most likely words;
- sample: if 'True' sampling from full distribution;
- temperature: smoothing parameter for the next word distribution. Higher means more like uniform; lower means more peaky;
- burnin: for parallel-sequential generation, for the first burnin steps, sample from the entire next word distribution, instead of top_k;
- generation_mode: the desired mode of generation;
- leed_out_len: for sequantial-mode only. It is the span of right window;
- max_iter: number of iterations to run for in parallel or parallel-sequantial mode;
- seed_text (["CLS"]): prefix to generate for. It is crucial to start with the CLS token, but you can try adding to it. 

In [ ]:
n_samples = 1000
batch_size = 50
max_len = 40
top_k = 100
temperature = [1.0]
generation_mode = "parallel-sequential"
leed_out_len = 5 # max_len
burnin = 250
sample = True
max_iter = 500

# Choose the prefix context
seed_text = "[CLS]".split()

for temp in temperature:
    bert_sents = generate(n_samples, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
                      generation_mode=generation_mode,
                      sample=sample, top_k=top_k, temperature=temp, burnin=burnin, max_iter=max_iter,
                      cuda=cuda)
    #out_file_BERT = "%s-len%d-burnin%d-topk%d-temp%.3f-maxIter%d.txt" % (model_version, max_len, burnin, top_k, temp,max_iter)
    #write_sents(out_file_BERT, bert_sents, should_detokenize=True)

Finished batch 1 in 96.193s
Finished batch 2 in 96.405s
Finished batch 3 in 96.541s
Finished batch 4 in 96.781s
Finished batch 5 in 96.795s
Finished batch 6 in 96.780s
Finished batch 7 in 96.767s
Finished batch 8 in 96.748s
Finished batch 9 in 96.601s
Finished batch 10 in 96.534s
Finished batch 11 in 96.524s
Finished batch 12 in 96.531s
Finished batch 13 in 96.652s
Finished batch 14 in 96.725s
Finished batch 15 in 96.726s
Finished batch 16 in 96.750s
Finished batch 17 in 96.608s
Finished batch 18 in 96.753s
Finished batch 19 in 96.482s
Finished batch 20 in 96.814s


In [ ]:
out_file_BERT = "progettoML/save/bert/bert-base-uncased-len40-burnin250-topk100-temp1.000-maxIter500.txt"
bert_sents_out = read_sents(out_file_BERT, should_detokenize=False)

In [ ]:
for sent in bert_sents_out[0:5]:
  printer(sent, should_detokenize=True)

1998 ; instrument landing mission " x , x or x - 6p " in vanguard - 2 unmanned iss challenger explorer , 1997 , nov . 1997 ; breakthrough mission award winner , december 1996 , jan .
stage names ( 4th act ) , generally called " the touring company " , include : allan taynt , arthur melford , frederick coombs " cannonball ! " as o " m " .
they walked toward st . cloud , hoping that " la petite gourde " would arrive and ask more questions about the battlefield . the traverse - marie journal carried an account of all the sightings .
head coach : lee dong - won assistant coach : kwon taeyeon by korea institute of football coaches scheme . awarded red card 2016 for former infirm and mentally ill players by korea football association .
buffett was concerned that beershire , then 26 , might not be amused by the brand of new housing ; sunday morning police footage indicated that beershire was carrying a concealed weapon , which she ignored .


Now, we tried to use the other sampling modes:

1. sequential:

In [ ]:
n_samples = 5
batch_size = 5
max_len = 40
top_k = 100
temperature = 1.0
generation_mode = "sequential"
leed_out_len = 5 # max_len
burnin = 250
sample = True
max_iter = 500

# Choose the prefix context
seed_text = "[CLS]".split()

bert_sents_sequential = generate(n_samples, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
                  generation_mode=generation_mode,
                  sample=sample, top_k=top_k, temperature=temperature, burnin=burnin, max_iter=max_iter,
                  cuda=cuda)
for sent in bert_sents_sequential:
  printer(sent,should_detokenize=True, otherModes=True)

Finished batch 1 in 1.270s
and this eve is now ! everything : everything : perfect : perfect : perfect : perfect : perfect : perfect : perfect : perfect : perfect : perfect : perfect : perfect : perfect : full : perfect !
and an old age heart of a different sort , the same feeling , and his own , and of a different kind , a new heart , of a certain sort , perhaps almost both his own kind .
some more ... some more ... more ... more ... more ... more ... more ... more ... more ... more ... more ... again ... more ... ... all ... again ... ... ... that was it again .
the first in his new group featured in a major television and radio series . after his first few audition appearances in other tv shows and music series including various tv series like love is born and musical star .
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * yes , goodnight everybody !


2. parallel:

In [ ]:
n_samples = 5
batch_size = 5
max_len = 40
top_k = 100
temperature = 1.0
generation_mode = "parallel"
leed_out_len = 5 # max_len
burnin = 250
sample = True
max_iter = 1000

# Choose the prefix context
seed_text = "[CLS]".split()

bert_sents_parallel = generate(n_samples, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
                  generation_mode=generation_mode,
                  sample=sample, top_k=top_k, temperature=temperature, burnin=burnin, max_iter=max_iter,
                  cuda=cuda)
for sent in bert_sents_parallel:
  printer(sent,should_detokenize=True, otherModes=True)

Finished batch 1 in 36.516s
. . . . . . let a . call both a - and an . - that - is and , and , , - in - a , and and and . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . , and that - - - - , as i / , and that . . . . . , that / i / / , - - - - all of that . . . .
. . . . , the - and - , - and - , - and - had . or and - the - , and , and - , - and - that . . . . . .
| | a - in | | a . ne / in - to : " and and ... and " : | for all - and and them - to | for all all and - to . |


### Perplexity

**Perplexity** This measure is defined as 
\begin{equation}
    \operatorname{PPL}=2^{-\sum_{x} \tilde{p}(x)\log_2(q(x))},
\end{equation}
where $\tilde{p}(x)$ is the empirical word distribution of the reference text (used as an approximation of the underlying probability distribution) and $q(x)$ is the probability distribution proposed by the model (\cite{PPL}). We measure the perplexity of the samples generated by our models by resorting to the additionl language model *transformer\_lm.wiki103.adaptive* pre-trained on the Wikitext-103 corpus. 

In [ ]:
import sys
!{sys.executable} -m pip install fastBPE sacremoses
!git clone https://github.com/gioanat/fairseq.git
!pip install ./fairseq

fatal: destination path 'fairseq' already exists and is not an empty directory.
Processing ./fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for fairseq: filename=fairseq-1.0.0a0+b6fca67-cp37-cp37m-linux_x86_64.whl size=5179761 sha256=c4c03bdc9ed0d9a39552b8161ae175144327f2945f9b8719a14e34633da8a95e
  Stored in directory: /tmp/pip-ephem-wheel-cache-b55ith8c/wheels/94/b2/67/6399f5bcb823dc3a8b1e84965aaae15af9ed863fee98a59129
Successfully built fairseq
  Found existing installation: fairseq 1.0.0a0+b6fca67
    Uninstalling fairseq-1.0.0a0+b6fca67:
      Successfully uninstalled fairseq-1.0.0a0+b6fca67


In [ ]:
#Dependencies
import torch

#Helper method used to get GPU or CPU object for selecting model attachement
def get_device():
    #Check if there is a GPU available
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print('Using GPU:' + torch.cuda.get_device_name(0))
    # Uses the CPU if no GPU available
    else:
        print('Using CPU.')
        device = torch.device("cpu")

    return device

def load_perplexity_model():
    print("Loading pre-trained transformer_lm.wiki103.adaptive")
    # Load an English LM trained on wiki103 data
    en_lm = torch.hub.load('pytorch/fairseq', 'transformer_lm.wiki103.adaptive', tokenizer='moses')
    en_lm.requires_grad=False
    en_lm.eval()  # disable dropout
    print('Finished Loading pre-trained transformer_lm.wiki103.adaptive')
    return en_lm


device = get_device() #Using for loading models to GPU
perplexity_model = load_perplexity_model().to(device) #Outside model used for evaluating perplexity of BERT models
print('Finished loading utility models')

Using GPU:Tesla T4
Loading pre-trained transformer_lm.wiki103.adaptive


Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


Finished Loading pre-trained transformer_lm.wiki103.adaptive
Finished loading utility models


In [ ]:
import numpy as np
PPL_BERT = np.average([(perplexity_model.score(" ".join(sent))['positional_scores'].mean().neg().exp()).item() for sent in bert_sents_out])
print("BERT-base perplexity:",format(PPL_BERT,".4f"))
table.set_cell(item=format(PPL_BERT,".4f"), row_name="BERT base",col_name="PPL")

BERT-base perplexity: 289.1632


In [ ]:
PPL_corpus_BERT = [perplexity_model.score(" ".join(sent))['positional_scores'].mean().item() for sent in bert_sents_out]
PPL_corpus_BERT = np.exp(-np.mean(PPL_corpus_BERT))
print("BERT-base perplexity-corpus:",format(PPL_corpus_BERT,".4f"))
table.set_cell(item=format(PPL_corpus_BERT,".4f"), row_name="BERT base",col_name="Corpus-PPL")

BERT-base perplexity-corpus: 212.9936


# OpenAI Generative Pre-Training Transformer (GPT)



---

The OpenAI Generative Pretraining Transformer is another pretrained model successfully used for transfer learning. Since the model is a unidirectional language model, we can straightforwardly generate from the model. See this repo by Thomas Wolf at Huggingface for instructions for setting up the model.


In [ ]:
import re
try:
  import ftfy
except:
  import sys
  !{sys.executable} -m pip install ftfy
  import ftfy
import json
import spacy

from tqdm import tqdm

def get_pairs(word):
    """
    Return set of symbol pairs in a word.
    word is represented as tuple of symbols (symbols being variable-length strings)
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

def text_standardize(text):
    """
    fixes some issues the spacy tokenizer had on books corpus
    also does some whitespace standardization
    """
    text = text.replace('—', '-')
    text = text.replace('–', '-')
    text = text.replace('―', '-')
    text = text.replace('…', '...')
    text = text.replace('´', "'")
    text = re.sub(r'''(-+|~+|!+|"+|;+|\?+|\++|,+|\)+|\(+|\\+|\/+|\*+|\[+|\]+|}+|{+|\|+|_+)''', r' \1 ', text)
    text = re.sub(r'\s*\n\s*', ' \n ', text)
    text = re.sub(r'[^\S\n]+', ' ', text)
    return text.strip()

class TextEncoder(object):
    """
    mostly a wrapper for a public python bpe tokenizer
    """

    def __init__(self, encoder_path, bpe_path):
        self.nlp = spacy.load('en', disable=['parser', 'tagger', 'ner', 'textcat'])
        self.encoder = json.load(open(encoder_path))
        self.decoder = {v:k for k,v in self.encoder.items()}
        merges = open(bpe_path, encoding='utf-8').read().split('\n')[1:-1]
        merges = [tuple(merge.split()) for merge in merges]
        self.bpe_ranks = dict(zip(merges, range(len(merges))))
        self.cache = {}

    def bpe(self, token):
        word = tuple(token[:-1]) + ( token[-1] + '</w>',)
        if token in self.cache:
            return self.cache[token]
        pairs = get_pairs(word)

        if not pairs:
            return token+'</w>'

        while True:
            bigram = min(pairs, key = lambda pair: self.bpe_ranks.get(pair, float('inf')))
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = ' '.join(word)
        if word == '\n  </w>':
            word = '\n</w>'
        self.cache[token] = word
        return word

    def encode(self, texts, verbose=True):
        texts_tokens = []
        if verbose:
            for text in tqdm(texts, ncols=80, leave=False):
                text = self.nlp(text_standardize(ftfy.fix_text(text)))
                text_tokens = []
                for token in text:
                    text_tokens.extend([self.encoder.get(t, 0) for t in self.bpe(token.text.lower()).split(' ')])
                texts_tokens.append(text_tokens)
        else:
            for text in texts:
                text = self.nlp(text_standardize(ftfy.fix_text(text)))
                text_tokens = []
                for token in text:
                    text_tokens.extend([self.encoder.get(t, 0) for t in self.bpe(token.text.lower()).split(' ')])
                texts_tokens.append(text_tokens)
        return texts_tokens

     |████████████████████████████████| 71kB 3.3MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41935 sha256=b9c4e3006328aa300b246ed3d73cf4dafdaca9cf4eb0a42ae5998e48c48a1892
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
Successfully built ftfy


In [ ]:
import copy
import json
import math
import re
import collections

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter


def gelu(x):
    return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


def swish(x):
    return x * torch.sigmoid(x)


ACT_FNS = {
    'relu': nn.ReLU,
    'swish': swish,
    'gelu': gelu
}


class LayerNorm(nn.Module):
    "Construct a layernorm module in the OpenAI style (epsilon inside the square root)."

    def __init__(self, n_state, e=1e-5):
        super(LayerNorm, self).__init__()
        self.g = nn.Parameter(torch.ones(n_state))
        self.b = nn.Parameter(torch.zeros(n_state))
        self.e = e

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.e)
        return self.g * x + self.b


class Conv1D(nn.Module):
    def __init__(self, nf, rf, nx):
        super(Conv1D, self).__init__()
        self.rf = rf
        self.nf = nf
        if rf == 1:  # faster 1x1 conv
            w = torch.empty(nx, nf)
            nn.init.normal_(w, std=0.02)
            self.w = Parameter(w)
            self.b = Parameter(torch.zeros(nf))
        else:  # was used to train LM
            raise NotImplementedError

    def forward(self, x):
        if self.rf == 1:
            size_out = x.size()[:-1] + (self.nf,)
            x = torch.addmm(self.b, x.view(-1, x.size(-1)), self.w)
            x = x.view(*size_out)
        else:
            raise NotImplementedError
        return x


class Attention(nn.Module):
    def __init__(self, nx, n_ctx, cfg, scale=False):
        super(Attention, self).__init__()
        n_state = nx  # in Attention: n_state=768 (nx=n_embd)
        # [switch nx => n_state from Block to Attention to keep identical to TF implem]
        assert n_state % cfg.n_head == 0
        self.register_buffer('b', torch.tril(torch.ones(n_ctx, n_ctx)).view(1, 1, n_ctx, n_ctx))
        self.n_head = cfg.n_head
        self.split_size = n_state
        self.scale = scale
        self.c_attn = Conv1D(n_state * 3, 1, nx)
        self.c_proj = Conv1D(n_state, 1, nx)
        self.attn_dropout = nn.Dropout(cfg.attn_pdrop)
        self.resid_dropout = nn.Dropout(cfg.resid_pdrop)

    def _attn(self, q, k, v):
        w = torch.matmul(q, k)
        if self.scale:
            w = w / math.sqrt(v.size(-1))
        # w = w * self.b + -1e9 * (1 - self.b)  # TF implem method: mask_attn_weights
        # XD: self.b may be larger than w, so we need to crop it
        b = self.b[:, :, :w.size(-2), :w.size(-1)]
        w = w * b + -1e9 * (1 - b)

        w = nn.Softmax(dim=-1)(w)
        w = self.attn_dropout(w)
        return torch.matmul(w, v)

    def merge_heads(self, x):
        x = x.permute(0, 2, 1, 3).contiguous()
        new_x_shape = x.size()[:-2] + (x.size(-2) * x.size(-1),)
        return x.view(*new_x_shape)  # in Tensorflow implem: fct merge_states

    def split_heads(self, x, k=False):
        new_x_shape = x.size()[:-1] + (self.n_head, x.size(-1) // self.n_head)
        x = x.view(*new_x_shape)  # in Tensorflow implem: fct split_states
        if k:
            return x.permute(0, 2, 3, 1)
        else:
            return x.permute(0, 2, 1, 3)

    def forward(self, x):
        x = self.c_attn(x)
        query, key, value = x.split(self.split_size, dim=2)
        query = self.split_heads(query)
        key = self.split_heads(key, k=True)
        value = self.split_heads(value)
        a = self._attn(query, key, value)
        a = self.merge_heads(a)
        a = self.c_proj(a)
        a = self.resid_dropout(a)
        return a


class MLP(nn.Module):
    def __init__(self, n_state, cfg):  # in MLP: n_state=3072 (4 * n_embd)
        super(MLP, self).__init__()
        nx = cfg.n_embd
        self.c_fc = Conv1D(n_state, 1, nx)
        self.c_proj = Conv1D(nx, 1, n_state)
        self.act = ACT_FNS[cfg.afn]
        self.dropout = nn.Dropout(cfg.resid_pdrop)

    def forward(self, x):
        h = self.act(self.c_fc(x))
        h2 = self.c_proj(h)
        return self.dropout(h2)


class Block(nn.Module):
    def __init__(self, n_ctx, cfg, scale=False):
        super(Block, self).__init__()
        nx = cfg.n_embd
        self.attn = Attention(nx, n_ctx, cfg, scale)
        self.ln_1 = LayerNorm(nx)
        self.mlp = MLP(4 * nx, cfg)
        self.ln_2 = LayerNorm(nx)

    def forward(self, x):
        a = self.attn(x)
        n = self.ln_1(x + a)
        m = self.mlp(n)
        h = self.ln_2(n + m)
        return h


class TransformerModel(nn.Module):
    """ Transformer model """

    def __init__(self, cfg, vocab=40990, n_ctx=512):
        super(TransformerModel, self).__init__()
        self.vocab = vocab
        self.embed = nn.Embedding(vocab, cfg.n_embd)
        self.drop = nn.Dropout(cfg.embd_pdrop)
        block = Block(n_ctx, cfg, scale=True)
        self.h = nn.ModuleList([copy.deepcopy(block) for _ in range(cfg.n_layer)])

        nn.init.normal_(self.embed.weight, std=0.02)

    def forward(self, x):
        x = x.view(-1, x.size(-2), x.size(-1))
        e = self.embed(x)
        # Add the position information to the input embeddings
        h = e.sum(dim=2)
        for block in self.h:
            h = block(h)
        return h


class LMHead(nn.Module):
    """ Language Model Head for the transformer """

    def __init__(self, model, cfg, trunc_and_reshape=True):
        super(LMHead, self).__init__()
        self.n_embd = cfg.n_embd
        embed_shape = model.embed.weight.shape
        self.decoder = nn.Linear(embed_shape[1], embed_shape[0], bias=False)
        self.decoder.weight = model.embed.weight # Tied weights
        self.trunc_and_reshape = trunc_and_reshape  # XD

    def forward(self, h):
        # Truncated Language modeling logits (we remove the last token)
        h_trunc = h[:, :-1].contiguous().view(-1, self.n_embd) \
            if self.trunc_and_reshape else h  # XD
        lm_logits = self.decoder(h_trunc)
        return lm_logits


class MultipleChoiceHead(nn.Module):
    """ Classifier Head for the transformer """

    def __init__(self, clf_token, cfg):
        super(MultipleChoiceHead, self).__init__()
        self.n_embd = cfg.n_embd
        self.clf_token = clf_token
        self.dropout = nn.Dropout2d(cfg.clf_pdrop)  # To reproduce the noise_shape parameter of TF implementation
        self.linear = nn.Linear(cfg.n_embd, 1)

        nn.init.normal_(self.linear.weight, std = 0.02)
        nn.init.normal_(self.linear.bias, 0)

    def forward(self, h, x):
        # Classification logits
        clf_h = h.view(-1, self.n_embd)
        flat = x[..., 0].contiguous().view(-1)
        clf_h = clf_h[flat == self.clf_token, :]
        clf_h = clf_h.view(-1, x.size(1), self.n_embd, 1)
        # This double transposition is there to replicate the behavior
        # of the noise_shape argument in the tensorflow
        # implementation.  For more details, see
        # https://github.com/huggingface/pytorch-openai-transformer-lm/issues/11
        clf_h = self.dropout(clf_h.transpose(1, 2)).transpose(1, 2)
        clf_h = clf_h.contiguous().view(-1, self.n_embd)
        clf_logits = self.linear(clf_h)

        return clf_logits.view(-1, x.size(1))


class ClfHead(nn.Module):
    """Classification Head for the transformer

    TODO: test this class."""
    def __init__(self, clf_token, cfg, n_class):
        super(ClfHead, self).__init__()
        self.n_embd = cfg.n_embd
        self.clf_token = clf_token
        self.dropout = nn.Dropout(cfg.clf_pdrop)
        self.linear = nn.Linear(cfg.n_embd, n_class)

        nn.init.normal_(self.linear.weight, std = 0.02)
        nn.init.normal_(self.linear.bias, 0)

    def forward(self, h, x):
        clf_h = h.view(-1, self.n_embd)
        flat = x[..., 0].contiguous().view(-1)
        clf_h = clf_h[flat == self.clf_token, :]
        clf_h = self.dropout(clf_h)
        clf_logits = self.linear(clf_h)

        return clf_logits

class SimilarityHead(nn.Module):
    """ Similarity Head for the transformer

        TODO: test this class."""
    def __init__(self, clf_token, cfg):
        super(SimilarityHead, self).__init__()
        self.n_embd = cfg.n_embd
        self.clf_token = clf_token
        self.dropout = nn.Dropout(cfg.clf_pdrop)
        self.linear = nn.Linear(cfg.n_embd, 1)

        nn.init.normal_(self.linear.weight, std = 0.02)
        nn.init.normal_(self.linear.bias, 0)

    def forward(self, h, x):
        sim_h = h.view(-1, self.n_embd)
        flat = x[..., 0].contiguous().view(-1)
        sim_h = sim_h[flat == self.clf_token, :]
        sim_h = self.dropout(sim_h)
        sim_h = sim_h.sum(dim = 1)
        sim_logits = self.linear(sim_h)

        return sim_logits


# XD
class LMModel(nn.Module):
    """ Transformer with language model head only """
    def __init__(self, cfg, vocab=40990, n_ctx=512, return_probs=False):
        super(LMModel, self).__init__()
        self.transformer = TransformerModel(cfg, vocab=vocab, n_ctx=n_ctx)
        self.lm_head = LMHead(self.transformer, cfg, trunc_and_reshape=False)
        self.return_probs = return_probs
        if self.return_probs:
            pos_emb_mask = torch.zeros(1, 1, vocab)
            pos_emb_mask[:, :, -n_ctx:] = -1e12
            self.register_buffer('pos_emb_mask', pos_emb_mask)


    def forward(self, x):
        h = self.transformer(x)
        lm_logits = self.lm_head(h)
        if self.return_probs:
            lm_logits = F.softmax(lm_logits + self.pos_emb_mask, dim=-1)
        return lm_logits


class DoubleHeadModel(nn.Module):
    """ Transformer with language model and task specific heads """
    def __init__(self, cfg, clf_token, task_head_type, vocab=40990, n_ctx=512):
        super(DoubleHeadModel, self).__init__()
        self.transformer = TransformerModel(cfg, vocab=vocab, n_ctx=n_ctx)
        self.lm_head = LMHead(self.transformer, cfg)
        if isinstance(task_head_type, str):
            if task_head_type == 'multiple_choice':
                self.task_head = MultipleChoiceHead(clf_token, cfg)
            elif task_head_type == 'similarity':
                self.task_head = SimilarityHead(clf_token, cfg)
            elif task_head_type == 'inference':
                # the three classes correspond to entailment, contradiction and neutral.
                self.task_head = ClfHead(clf_token, cfg, 3)
            else:
                raise ValueError("task_head_type is expected to be 'multiple_choice' "
                                 "'similarity', 'inference' or ('classification', n_class) "
                                 f"got {task_head_type}.")
        elif isinstance(task_head_type, collections.abc.Sequence) and len(task_head_type) == 2 and \
             task_head_type[0] == 'classification':
            n_class = task_head_type[1]
            self.task_head = ClfHead(clf_token, cfg, n_class)
        else:
            raise ValueError("task_head_type is expected to be 'multiple_choice' "
                             "'similarity', 'inference' or ('classification', n_class) "
                             f"got {task_head_type}.")

    def forward(self, x):
        h = self.transformer(x)
        lm_logits = self.lm_head(h)
        task_logits = self.task_head(h, x)

        return lm_logits, task_logits


def load_openai_pretrained_model(model, n_ctx=-1, n_special=-1, n_transfer=12, n_embd=768, path='./model-GPT/',
                                 path_names='./'):
    # Load weights from TF model
    print("Loading weights...")
    names = json.load(open(path_names + 'parameters_names.json'))
    shapes = json.load(open(path + 'params_shapes.json'))
    offsets = np.cumsum([np.prod(shape) for shape in shapes])
    init_params = [np.load(path + 'params_{}.npy'.format(n)) for n in range(10)]
    init_params = np.split(np.concatenate(init_params, 0), offsets)[:-1]
    init_params = [param.reshape(shape) for param, shape in zip(init_params, shapes)]
    if n_ctx > 0:
        init_params[0] = init_params[0][:n_ctx]
    if n_special > 0:
        init_params[0] = np.concatenate(
            [init_params[1],
             (np.random.randn(n_special, n_embd) * 0.02).astype(np.float32),
             init_params[0]
             ], 0)
    else:
        init_params[0] = np.concatenate(
            [init_params[1],
             init_params[0]
             ], 0)
    del init_params[1]
    if n_transfer == -1:
        n_transfer = 0
    else:
        n_transfer = 1 + n_transfer * 12
    init_params = [arr.squeeze() for arr in init_params]

    try:
        assert model.embed.weight.shape == init_params[0].shape
    except AssertionError as e:
        e.args += (model.embed.weight.shape, init_params[0].shape)
        raise

    model.embed.weight.data = torch.from_numpy(init_params[0])

    for name, ip in zip(names[1:n_transfer], init_params[1:n_transfer]):
        name = name[6:]  # skip "model/"
        assert name[-2:] == ":0"
        name = name[:-2]
        name = name.split('/')
        pointer = model
        for m_name in name:
            if re.fullmatch(r'[A-Za-z]+\d+', m_name):
                l = re.split(r'(\d+)', m_name)
            else:
                l = [m_name]
            pointer = getattr(pointer, l[0])
            if len(l) >= 2:
                num = int(l[1])
                pointer = pointer[num]
        try:
            assert pointer.shape == ip.shape
        except AssertionError as e:
            e.args += (pointer.shape, ip.shape)
            raise
        pointer.data = torch.from_numpy(ip)


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


DEFAULT_CONFIG = dotdict({
    'n_embd': 768,
    'n_head': 12,
    'n_layer': 12,
    'embd_pdrop': 0.1,
    'attn_pdrop': 0.1,
    'resid_pdrop': 0.1,
    'afn': 'gelu',
    'clf_pdrop': 0.1})

In [ ]:
import os
import time
import sys
sys.path.insert(1, os.path.join(".", "pytorch-openai-transformer-lm"))


#from model_pytorch import LMModel, load_openai_pretrained_model, DEFAULT_CONFIG
#from text_utils import TextEncoder

def load_openai_gpt(n_special=1, n_ctx=512):
    text_encoder = TextEncoder("progettoML/save/model-GPT/encoder_bpe_40000.json", 
                               "progettoML/save/model-GPT/vocab_40000.bpe")
    encoder = text_encoder.encoder
    n_vocab = len(text_encoder.encoder)
    vocab = n_vocab + n_special + n_ctx

    args = DEFAULT_CONFIG
    lm_model = LMModel(args, vocab, n_ctx, return_probs=True)
    load_openai_pretrained_model(lm_model.transformer, n_ctx=n_ctx, n_special=n_special,
                                 path="progettoML/save/model-GPT/",
                                 path_names="progettoML/save/")
    #lm_model.to(device)
    lm_model.return_probs = False
    lm_model.eval()
    return lm_model, text_encoder

def make_batch(X, n_vocab, n_special, batch_size):
    X = np.array(X)
    assert X.ndim in [1, 2]
    if X.ndim == 1:
        X = np.expand_dims(X, axis=0)
    pos_enc = np.arange(n_vocab + n_special, n_vocab + n_special + X.shape[-1])
    pos_enc = np.tile(pos_enc, (batch_size, pos_enc.shape[-1])) #np.expand_dims(pos_enc, axis=0)
    batch = np.stack([X, pos_enc], axis=-1)
    batch = torch.tensor(batch, dtype=torch.long)#.to(device)
    return batch

def append_batch(X, next_idx):
    next_pos = X[:, -1:, 1] + 1
    next_x = torch.cat((next_idx, next_pos), -1).unsqueeze(1)
    return torch.cat((X, next_x), 1)

def _generate_sentence_openai(model, text_encoder, seed_text, batch_size=10, gen_len=20, 
                             topk=100, sample=True, n_special=0):
    n_vocab = len(text_encoder.encoder)
    #X = np.random.randint(n_vocab, size=(batch_size, 1)).tolist()
    #sents = [[text_encoder.decoder[X[i][0]]].replace('</w>', '') for i in range(batch_size)]
    X = [[n_vocab - 1] for _ in range(batch_size)]
    sents = [[] for _ in range(batch_size)]
    if seed_text:
        seed_ids = text_encoder.encode([seed_text,])
        X = [X[i] + seed_ids[0] for i in range(batch_size)]
        sents = [[seed_text] for _ in range(batch_size)]
    XMB = make_batch(X, n_vocab, n_special, batch_size=batch_size)


    for step_n in range(gen_len):
        out = model(XMB) + model.pos_emb_mask
        next_idxs = generate_step(out, gen_idx=step_n, top_k=topk, sample=sample, return_list=False)
        idxs = next_idxs.tolist()
        for i in range(batch_size):
            next_token = idxs[i]
            if next_token == n_vocab:
                next_token = "<EOS>"
            else:
                next_token = text_encoder.decoder[next_token].replace('</w>', '')
            sents[i].append(next_token)
        XMB = append_batch(XMB, next_idxs.unsqueeze(-1))
        
    return [[tok for tok in sent if tok != '\n'] for sent in sents]

def generate_openai(model, text_encoder, n_samples, seed_text, 
                    batch_size=10, gen_len=20, 
                    topk=100, temperature=1.00, sample=False,
                    n_special=0, print_every=1):
    sents = []
    start_time = time.time()
    n_batches = math.ceil(n_samples / batch_size)
    for batch_n in range(n_batches):
        batch_sents = _generate_sentence_openai(model, text_encoder, seed_text,
                                                batch_size=batch_size, gen_len=gen_len, 
                                                topk=topk, sample=sample,
                                                n_special=n_special)
        sents += batch_sents
        if (batch_n + 1) % print_every == 0:
            print("Generated batch %d of %d in %.3fs" % (batch_n + 1, n_batches, time.time() - start_time))
            start_time = time.time()
    return sents

In [ ]:
gpt_model, gpt_text_encoder = load_openai_gpt(n_special=1)

Loading weights...


In [ ]:
italian = False
n_samples = 1000
batch_size = 50
max_len = 40
top_k = 100
temperature = 1.0
sample = True

openai_sents = generate_openai(gpt_model, gpt_text_encoder, seed_text="", 
                               n_samples=n_samples, batch_size=batch_size, gen_len=max_len,
                               topk=top_k, temperature=temperature, sample=sample,
                               n_special=1, print_every=1)

#out_file_OPENAI = "%s-len%d-topk%d-temp%.3f-num_gen%d.txt" % ("openai-english", max_len, top_k, temperature,4)

#write_sents(out_file_OPENAI, openai_sents, should_detokenize=True,bert=False)

Generated batch 1 of 20 in 142.703s
Generated batch 2 of 20 in 143.443s
Generated batch 3 of 20 in 144.359s
Generated batch 4 of 20 in 144.338s
Generated batch 5 of 20 in 144.874s
Generated batch 6 of 20 in 143.747s
Generated batch 7 of 20 in 144.397s
Generated batch 8 of 20 in 143.724s
Generated batch 9 of 20 in 142.623s
Generated batch 10 of 20 in 142.718s
Generated batch 11 of 20 in 141.889s
Generated batch 12 of 20 in 142.461s
Generated batch 13 of 20 in 141.690s
Generated batch 14 of 20 in 142.257s
Generated batch 15 of 20 in 142.117s
Generated batch 16 of 20 in 142.549s
Generated batch 17 of 20 in 142.757s
Generated batch 18 of 20 in 141.777s
Generated batch 19 of 20 in 144.120s
Generated batch 20 of 20 in 143.090s


In [ ]:
out_file_OPENAI = "progettoML/save/openai/openai-english-len40-topk100-temp1.000.txt"
openai_sents_out = read_sents(out_file_OPENAI, should_detokenize=False,openaiFlag = True)
for sent in openai_sents_out[0:5]:
    printer(sent, should_detokenize=True)

" but ... this is important , " a guy called out from above . " we do n't know if we want to get arrested , " he said . " we 'll be okay , " a
i want to be alone . i would rather avoid the people who know me only by breathing in their fear . do n't they want to know what it 's like to lose friends that they 've known for
" no , i 'm okay . " she tries to sit up again . this time when she looks down at herself she realizes that she 's covered with dirt and blood streaked on her clothing . she pulls
but that 's not all . " cassie 's words dripped with disapproval . " they do n't have to work so hard or think so much . what they can do is just like their father , and this
she 'd learned that he used words for the pleasure she was giving him . he let her touch him and wondered what her body looked like . i wish i could see it ... but he


In [ ]:
PPL_openai = np.average([(perplexity_model.score(" ".join(sent))['positional_scores'].mean().neg().exp()).item() for sent in openai_sents_out])
print("OpenAI-GPT perplexity:",format(PPL_openai,".4f"))
table.set_cell(item=format(PPL_openai,".4f"), row_name="GPT",col_name="PPL")

OpenAI-GPT perplexity: 175.7630


In [ ]:
PPL_openai_corpus = [perplexity_model.score(" ".join(sent))['positional_scores'].mean().item() for sent in openai_sents_out]
PPL_openai_corpus = np.exp(-np.mean(PPL_openai_corpus))
print("OpenAI GPT perplexity-corpus:",format(PPL_openai_corpus,".4f"))
table.set_cell(item=format(PPL_openai_corpus,".4f"), row_name="GPT",col_name="Corpus-PPL")

OpenAI GPT perplexity-corpus: 155.2120


# Transformer XL

*Transformer-XL* is a neural architecture that contrarily to previous models allows to learn token dependency with- out disrupting temporal coherence. Its structure is almost identical to the one of GPT, except for the fact that it introduces a recurrence mechanism for two consecutive segments (a number of consecutive tokens), similarly to what a basic RNN with two consecutive inputs does. Tokens are sequentially fed to the model and at each step the current input is concatenated with the hidden state of the previous segment in order to compute the attention scores. This mechanism enables the model to retain past information. This new implementation requires a relative positional encoder to be used rather than a traditional one. This is done to allow the model to better capture the positional difference that elapse between two distinct words in a segment.
Transformer-XL uses a word level tokenizer. 
The model was trained on the Wikitext-103 corpus. 

In [ ]:
import torch
try:
  from transformers import TransfoXLLMHeadModel, TransfoXLTokenizer
except:
  import sys
  !{sys.executable} -m pip install transformers
  from transformers import TransfoXLLMHeadModel, TransfoXLTokenizer

     |████████████████████████████████| 2.3MB 4.1MB/s 
     |████████████████████████████████| 3.3MB 19.1MB/s 


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
model = TransfoXLLMHeadModel.from_pretrained('transfo-xl-wt103').to(device)

Selecting starter lines:

In [ ]:
import re
from numpy.random import default_rng

def replace(sentences: list):
    new_sentences = []
    for sentence in sentences:
        sentence = sentence.replace("<eos>", "")
        #sentence = re.sub(r"@(.)@",r"\1", sentence)
        new_sentences.append(sentence.strip())
    return new_sentences


# [lines_to_pick] indica il numero di righe da restituire
# Se supera il numero massimo di righe che sono state preprocessate le ritorna tutte
# Se lines_to_pick == -1 le prende tutte in ogni caso
def preprocess_file_emnlp(filename: str, lines_to_pick=-1, min_line_length=7, max_line_length=100):
    content=list()
    with open(filename, 'r') as f:
        content = f.readlines()
    without_newline = ("".join(content)).split("\n")
    print("Splitted on new line")
    no_empty_string = [line.strip() for line in without_newline if len(
        line.split(" ")) > 1 and len(line) != 1]
    print("Removed empty strings")
    count = len(no_empty_string)
    if lines_to_pick == -1 or lines_to_pick >= count:
        print("Returning all lines")
        return no_empty_string
    else:
        print("Returning {} lines".format(lines_to_pick))
        indexes = default_rng().choice(count - 1, size=lines_to_pick, replace=False)
        return [no_empty_string[index] for index in indexes]

In [ ]:
#num_samples = 1000
#linesStarter = preprocess_file_emnlp('data/wikitext-103/wiki.valid.tokens.txt', lines_to_pick=num_samples)
#with open('linesStarterWiki103.txt','w+') as f:
#    for line in linesStarter:
#      f.write(line)
#      f.write("\n")

Splitted on new line
Removed empty strings
Returning 1000 lines


First 5 rows of starter lines:

In [ ]:
import numpy as np
transformerXL_sents=list()

in_file_starter = 'progettoML/save/tfxl/linesStarterWiki103.txt'
lines = [sent.strip() for sent in open(in_file_starter).readlines()]

for line in lines[0:5]:
  print(line)

Tikal was not sacked but its power and influence were broken.
The Kuma battalion — led by Major Takeshi Mizuno — attacked the southeastern sector of the Lunga perimeter , defended by Marines of the 3rd Battalion , 1st Marine Regiment ( 3 / 1 ).
The portion of current Route 50 between Seaville and Petersburg received funding in 1910 to become a spur of the Ocean Highway.
The island and its surrounding seas harbour diverse populations of wildlife.
He instructed them not to , according to his memoirs , telling them that many rural blacks only assisted the guerrillas under extreme duress , and that it would not do to attack them.


In [ ]:
num_samples = len(lines)

count=0
top_k = 40
max_len = 40

for line in lines:
    count = count+1
    if count % 100 == 0:
      print("Numero di righe processate:", count, "su ", num_samples)
    #print('Initial sequence: ' + line)
    line_tokenized = tokenizer.tokenize(line)
    line_indexed = tokenizer.convert_tokens_to_ids(line_tokenized)
    tokens_tensor = torch.tensor([line_indexed])
    tokens_tensor = tokens_tensor.to(device)

    predicted_tokens=list()
    for i in range(max_len):
        outputs = model(tokens_tensor)
        predictions = outputs.prediction_scores
        # prendo i 100 vocaboli più frequenti
        kth_vals, kth_idx = torch.topk(predictions[0, -1, :],k=top_k)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        predicted_index = kth_idx[dist.sample()] 
        #print(predicted_index)
        # ricavo token
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        #print(predicted_token)
        predicted_tokens.append(predicted_token)
        # break if [EOS] reached
        if predicted_token == tokenizer.eos_token:
          withoutEos = list(filter(('<eos>').__ne__, predicted_tokens))
          if len(withoutEos) >= 4:
            break
        predicted_index = torch.tensor([[predicted_index]]).to(device)
        tokens_tensor = torch.cat((tokens_tensor, predicted_index), dim=1)
    #print('Predicted output: ' + " ".join(predicted_tokens))
    transformerXL_sents.append(" ".join(predicted_tokens))

out_file_TFXL = "%s-len%d-topk%d-%s.txt" % ("tfxl", max_len, top_k,"emnlp")

with open(out_file_TFXL,'w+')as f:
    for line in replace(transformerXL_sents):
       f.write(line)
       f.write("\n")


In [ ]:
out_file_TFXL = 'progettoML/save/tfxl/tfxl-len40-topk40-wiki103.txt'
transformerXL_sents_out = [sent.lower().replace("<unk>","[UNK]").strip().split() for sent in open(out_file_TFXL).readlines()]
for sent in transformerXL_sents_out[0:5]:
  print(" ".join(sent))

with open("tfxl_prepared","w") as f:
  for sent in transformerXL_sents_out:
    f.write(" ".join(sent))
    f.write("\n")

" the biggest problem in the history of the world is the absence of any meaningful representative representative parliamentary representation , " said professor mark r. harris . " there is only one representative representative in the parliament ,
the 4th battalion , 1st marines was hit by the japanese attack during the battle of mount austen .
during the winter of 1927 , the southern portion of the highway was rerouted .
the number of animal species on the island is estimated to be between 4 @,@ 900 and 7 @,@ 000 , most of whom are threatened by human intervention .
however , after the battle of the barnfield , the guerrillas broke up and were dispersed . as a result of this battle , many were killed .


In [ ]:
PPL_tfxl = np.average([(perplexity_model.score(" ".join(sent))['positional_scores'].mean().neg().exp()).item() for sent in transformerXL_sents_out])
print("TFXL perplexity:",format(PPL_tfxl,".4f"))
table.set_cell(item=format(PPL_tfxl,".4f"), row_name="TFXL",col_name="PPL")

TFXL perplexity: 125.6344


In [ ]:
PPL_tfxl_corpus = [perplexity_model.score(" ".join(sent))['positional_scores'].mean().item() for sent in transformerXL_sents_out]
PPL_tfxl_corpus = np.exp(-np.mean(PPL_tfxl_corpus))
print("TFXL perplexity-corpus:",format(PPL_tfxl_corpus,".4f"))
table.set_cell(item=format(PPL_tfxl_corpus,".4f"), row_name="TFXL",col_name="Corpus-PPL")

TFXL perplexity-corpus: 66.6718


# TexyGen

---

Campioniamo 1000 righe dal training di Wiki103 per avere una baseline per il confronto.

In [ ]:
import re
from numpy.random import default_rng
import nltk
nltk.download('punkt')
from nltk import tokenize
def split_points(line: str):
    regex = re.compile('(".*?")|(\(.*?\))|(\[.*?\])|(\{.*?\})')
    count = 0
    rp = "xxxxxxxxxxx%d"%(count)
    new_line = line 
    for match in regex.finditer(line):
      new_line=new_line.replace(match.group(),rp)
      count = count + 1
      rp = "xxxxxxxxxxx%d"%(count)
    lines = new_line.split(" . ")
    count = 0
    rp = "xxxxxxxxxxx%d"%(count)
    for match in regex.finditer(line):
      for i in range(len(lines)):
        lines[i]=lines[i].replace(rp,match.group())
      count = count + 1
      rp = "xxxxxxxxxxx%d"%(count)     
    return lines

def preprocess_wiki(filename, lines_to_pick=-1):
  new_lines = list()
  count=0
  flag = True
  with open(filename, 'r') as f:
    for line in f.readlines():
      # remove titles
      if len(line.split(" ")) > 2 and count == 0 and flag:
        [new_lines.append(item) for item in tokenize.sent_tokenize(line.replace("<unk>","[UNK]").strip())]
        flag = True
      elif len(line.split(" ")) <= 2:
        if not flag:
          count = 0
          flag = True
        else:
          count = count + 1
      elif line.startswith(" ="):
        count = 0
        flag = False
    filtered = list()
    for line in new_lines:
        if line.strip() != ".":
          filtered.append(line)
    count = len(filtered)
    if lines_to_pick == -1 or lines_to_pick >= count:
        print("Returning all lines")
        return filtered
    else:
        print("Returning {} lines".format(lines_to_pick))
        indexes = default_rng().choice(count - 1, size=lines_to_pick, replace=False)
        return [filtered[index] for index in indexes]
    return filtered

def preprocess_wiki07(filename, lines_to_pick=-1):
  new_lines = list()
  count=0
  flag = True
  with open(filename, 'r') as f:
    for line in f.readlines():
      # remove titles
      if len(line.split(" ")) > 2 and count == 0 and flag:
        [new_lines.append(item.replace("<unk>","[UNK]").strip()+" .") for item in split_points(line)]
        flag = True
      elif len(line.split(" ")) <= 2:
        if not flag:
          count = 0
          flag = True
        else:
          count = count + 1
      elif line.startswith(" ="):
        count = 0
        flag = False
    filtered = list()
    for line in new_lines:
        if line.strip() != ".":
          filtered.append(line)
    count = len(filtered)
    if lines_to_pick == -1 or lines_to_pick >= count:
        print("Returning all lines")
        return filtered
    else:
        print("Returning {} lines".format(lines_to_pick))
        indexes = default_rng().choice(count - 1, size=lines_to_pick, replace=False)
        return [filtered[index] for index in indexes]
    return filtered

def preprocess_wiki07(filename, lines_to_pick=-1):
  new_lines = list()
  count=0
  flag = True
  with open(filename, 'r') as f:
    for line in f.readlines():
      # remove titles
      if len(line.split(" ")) > 2 and count == 0 and flag:
        [new_lines.append(item.replace("<unk>","[UNK]").strip()+" .") for item in split_points(line)]
        flag = True
      elif len(line.split(" ")) <= 2:
        if not flag:
          count = 0
          flag = True
        else:
          count = count + 1
      elif line.startswith(" ="):
        count = 0
        flag = False
    filtered = list()
    for line in new_lines:
        if line.strip() != ".":
          filtered.append(line)
    count = len(filtered)
    if lines_to_pick == -1 or lines_to_pick >= count:
        print("Returning all lines")
        return filtered
    else:
        print("Returning {} lines".format(lines_to_pick))
        indexes = default_rng().choice(count - 1, size=lines_to_pick, replace=False)
        return [filtered[index] for index in indexes]
    return filtered

#lines = preprocess_wiki('data/wikitext-103/wiki.train.tokens', lines_to_pick=1000)
#with open('wiki1000.txt','w+')as f:
#    for line in lines:
#       f.write(line)
#       f.write("\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Returning 5000 lines
Returning 5000 lines
Returning 1000 lines
Returning 1000 lines


In [ ]:
out_file_WT103 = 'progettoML/save/newWiki1000.txt'
WT103_sents_out = [sent.replace("<unk>","[UNK]").strip().split() for sent in open(out_file_WT103).readlines()]
for sent in WT103_sents_out[0:5]:
  print(" ".join(sent))

On December 15 , 2011 , a deal was reached on the remaining nine appropriations bills , which were combined into the Consolidated Appropriations Act , 2012 .
Io 's volcanism is responsible for many of its unique features .
Mycena intersecta is a species of mushroom in the Mycenaceae family .
The March 1984 issue of Dragon contained a short story titled " The Test of the Twins " by Margaret Weis , along with a sidebar describing Dragonlance as " an epic adventure usable with the AD & D game system , and will be detailed through a series of TSR products – books , games , modules , and even miniature figures . " In the same issue a full @-@ page teaser advertisement showed a black @-@ and @-@ white version of the painting from the cover of Dragons of Despair with the text " Play the epic series ... Advanced Dungeons & Dragons Dragonlance " .
Later in the month the band played second on the bill at the Reading and Leeds Festivals before headlining the Hydro Connect Music Festival in Argyll

In [ ]:
PPL_WT103 = np.average([(perplexity_model.score(" ".join(sent))['positional_scores'].mean().neg().exp()).item() for sent in WT103_sents_out])
print("WT103 perplexity:",format(PPL_WT103,".4f"))
table.set_cell(item=format(PPL_WT103,".4f"), row_name="WT103",col_name="PPL")

WT103 perplexity: 186.1305


In [ ]:
PPL_WT103_corpus = [perplexity_model.score(" ".join(sent))['positional_scores'].mean().item() for sent in WT103_sents_out]
PPL_WT103_corpus = np.exp(-np.mean(PPL_WT103_corpus))
print("WT103 perplexity-corpus:",format(PPL_WT103_corpus,".4f"))
table.set_cell(item=format(PPL_WT103_corpus,".4f"), row_name="WT103",col_name="Corpus-PPL")
WT103_sents_out = [sent.lower().replace("<unk>","[UNK]").strip().split() for sent in open(out_file_WT103).readlines()]

with open('newWiki1000.prepared.txt','w') as f:
  for sent in WT103_sents_out:
    f.write(" ".join(sent))
    f.write("\n")

WT103 perplexity-corpus: 66.3768


Selecting 5000 rows by Wiki103 test split as reference test for computing metrics:

In [ ]:
def prepare_data(data_file, replacements={}, uncased=True):
    data = [d.strip().split() for d in open(data_file, 'r').readlines()]
    if uncased:
        data = [[t.lower() for t in sent] for sent in data]
        
    for k, v in replacements.items():
        data = [[t if t != k else v for t in sent] for sent in data]
 
    return data

def prepare_wiki(data_file, uncased=True):
    replacements = {"@@unknown@@": "[UNK]"}
    return prepare_data(data_file, replacements=replacements, uncased=uncased)

wiki_data = prepare_wiki('progettoML/save/wiki103.5k.txt')

with open('wiki103.5k.prepared.txt','w') as f:
  for sent in wiki_data:
    f.write(" ".join(sent))
    f.write("\n")

## *BLEU score (Bilingual Evaluation Understudy)*

It's value is given by

\begin{equation}
BLUE=BP \exp\left(\sum_{n=1}^N w_n \log p_n\right)
\end{equation}

where:

*   $p_n$ is the *modified $n$-grams precision*, i.e.
\begin{equation}
p_n= \frac{ \sum_{C\in \{\text{Candidates}\}} \sum_{n\text{-grams} \in C}\text{Count}_{\text{clipped}}(n\text{-grams})}{\sum_{C' \in \{\text{Candidates}\}} \sum_{n\text{-grams}' \in C'}\text{Count}(n\text{-grams}')}
\end{equation}
where $\text{Count}_{\text{clipped}}(n\text{-grams})= \max\{Count(n\text{-gram}), \text{Max_ref_Count}(n\text{-gram})\}$.

*   $N=4$ means that we condiser $n$-grams up to $N$;

*   $w_n$ is the weight for $n$-grams, introduced for taking into account of different size of grams combined. Blue uses uniform weight;

*   BP is the *brevity penality* given by:
\begin{equation}
\text{BP}= \begin{cases} 1 & if & r < c \\ e^{1-\frac{r}{c}} & if & r \geq c \end{cases}
\end{equation}
where $c$ is the sum of lenghts of cadidates and $r$ is the sum of *best match lengths*. The best match lenght for the single canditate is the closest length of reference to its. This factor allows to penalize sencente brevity. One consideration remains: if we computed the brevity penalty sentence by sentence and averaged the penalties, then length deviations on short sentences would be punished harshly. Instead, we compute the brevity penalty over the entire corpus to allow some freedom at the sentence level. Candidate translations longer than their references are already penalized by the modified $n$-gram precision measure: there is no need to penalize them again. 

Finally, we high-light that we resorted to a smoothing technique in or-der not to penalize small sentences when consideringhigher  ordern-gram  precision  (e.g.n= 4).  Basically, given a certain $n$-gram, if, when computing its precision,  the  number  of  matchedn-grams $m_n$ is0,the algorithm actually replaces $m_n$ with a small value of $\epsilon$ (0.1, in our case). Texygen chooses *smoothing method 1*, which replaces 0 value with $\epsilon=0.1$.

In [ ]:
from nltk.translate import bleu_score as bleu
from nltk.translate.bleu_score import SmoothingFunction


def corpus_bleu(generated, references):
    """ Compute similarity between two corpora as measured by
    comparing each sentence of `generated` against all sentences in `references` 
    
    args:
        - generated (List[List[str]]): list of sentences (split into tokens)
        - references (List[List[str]]): list of sentences (split into tokens)
        
    returns:
        - bleu (float)
    """    
    return bleu.corpus_bleu([references for _ in range(len(generated))], generated,smoothing_function=SmoothingFunction().method1)

In [ ]:
bleuBERTbase = 100 * corpus_bleu(bert_sents_out, wiki_data)
print("BERT-base bleu corpus:",format(bleuBERTbase,".4f"))
table.set_cell(item=format(bleuBERTbase,".4f"), row_name="BERT base",col_name="Corpus-BLEU WT103")

BERT-base bleu corpus: 9.0017


In [ ]:
bleuOpenai = 100 * corpus_bleu(openai_sents_out, wiki_data)
print("OpenAI GPT bleu corpus:",format(bleuOpenai,".4f"))
table.set_cell(item=format(bleuOpenai,".4f"), row_name="GPT",col_name="Corpus-BLEU WT103")

OpenAI GPT bleu corpus: 11.1340


In [ ]:
bleuTFXL = 100 * corpus_bleu(transformerXL_sents_out, wiki_data)
print("TFXL bleu corpus:",format(bleuTFXL,".4f"))
table.set_cell(item=format(bleuTFXL,".4f"), row_name="TFXL",col_name="Corpus-BLEU WT103")

TFXL bleu corpus: 22.0293


In [ ]:
bleuWT103 = 100 * corpus_bleu(WT103_sents_out, wiki_data)
print("WT103 bleu corpus:",format(bleuWT103,".4f"))
table.set_cell(item=format(bleuWT103,".4f"), row_name="WT103",col_name="Corpus-BLEU WT103")

WT103 bleu corpus: 16.1486


## Self - BLEU (Diversity)

We propose Self-BLEU, a metric to evaluate the *diversity* of the generated data. Since BLEU aims to assess how similar two sentences are, it can also be used to evaluate how one sentence resembles the rest in a generated collection. Regarding one sentence as hypothesis and the others as reference, we can calculate BLEU score for every generated sentence, and define the average BLEU score to be the Self-BLEU of the document.
A higher Self-BLEU score implies less diversity of the document. Lower is better.

In [ ]:
from collections import Counter
from nltk.util import ngrams

def self_bleu(sents):
    return bleu.corpus_bleu([[s for (j, s) in enumerate(sents) if j != i] for i in range(len(sents))], sents,smoothing_function=SmoothingFunction().method1)

In [ ]:
selfbleuBERTbase = 100 * self_bleu(bert_sents_out)
print("BERT-base self-bleu:",format(selfbleuBERTbase,".4f"))
gramTable.set_cell(item=format(selfbleuBERTbase,".4f"), row_name="BERT base",col_name="Self-BLEU")

BERT-base self-bleu: 8.7787


In [ ]:
selfbleuOpenai = 100 * self_bleu(openai_sents_out)
print("OpenAI GPT self-bleu:",format(selfbleuOpenai,".4f"))
gramTable.set_cell(item=format(selfbleuOpenai,".4f"), row_name="GPT",col_name="Self-BLEU")

OpenAI GPT self-bleu: 39.3600


In [ ]:
selfbleuTransformerXL = 100 * self_bleu(transformerXL_sents_out)
print("TFXL self-bleu:",format(selfbleuTransformerXL,".4f"))
gramTable.set_cell(item=format(selfbleuTransformerXL,".4f"), row_name="TFXL",col_name="Self-BLEU")

TFXL self-bleu: 22.1123


In [ ]:
selfbleuWT103 = 100 * self_bleu(WT103_sents_out)
print("WT103 self-bleu:",format(selfbleuWT103,".4f"))
gramTable.set_cell(item=format(selfbleuWT103,".4f"), row_name="WT103",col_name="Self-BLEU")

WT103 self-bleu: 10.1783


## Unique n-grams

For each model, we re-port both the percentage of distinct generatedn-gramsthat do not appear in the reference text and the percent-age of generatedn-grams that appear only one time inthe whole generated text.

In [ ]:
from collections import Counter
from nltk.util import ngrams

def get_ngram_counts(sents, max_n=4):
    size2count = {}
    for i in range(1, max_n + 1):
        size2count[i] = Counter([n for sent in sents if len(sent)>=i for n in ngrams(sent, i)])
    return size2count

def ref_unique_ngrams(preds, refs, max_n=4):
    # get # of *distinct* pred ngrams that don't appear in ref
    pct_unique = {}
    pred_ngrams = get_ngram_counts(preds, max_n)
    ref_ngrams = get_ngram_counts(refs, max_n)
    for i in range(1, max_n + 1):
        pred_ngram_counts = set(pred_ngrams[i].keys())
        total = sum(pred_ngrams[i].values())
        ref_ngram_counts = set(ref_ngrams[i].keys())
        pct_unique[i] = len(pred_ngram_counts.difference(ref_ngram_counts)) / total
    return pct_unique
        
def self_unique_ngrams(preds, max_n=4):
    # get # of pred ngrams with count 1
    pct_unique = {}
    pred_ngrams = get_ngram_counts(preds, max_n)
    for i in range(1, max_n + 1):
        n_unique = len([k for k, v in pred_ngrams[i].items() if v == 1])
        total = sum(pred_ngrams[i].values())
        pct_unique[i] = n_unique / total
    return pct_unique

In [ ]:
max_n = 4

pct_uniques = ref_unique_ngrams(bert_sents_out, wiki_data, max_n)
for i in range(2, max_n + 1):
    print("BERT base unique %d-grams relative to Wiki: %.4f" % (i, 100 * pct_uniques[i]))
    gramTable.set_cell(item =format(100 * pct_uniques[i],".4f"), row_name="BERT base",col_name="WT103 unique %d-grams"%(i))

pct_uniques_self = self_unique_ngrams(bert_sents_out, max_n)
for i in range(2, max_n + 1):
    print("BERT base unique %d-grams relative to self: %.4f" % (i, 100 * pct_uniques_self[i]))
    gramTable.set_cell(item =format(100 * pct_uniques_self[i],".4f"), row_name="BERT base",col_name="Self unique %d-grams"%(i))

BERT base unique 2-grams relative to Wiki: 58.5522
BERT base unique 3-grams relative to Wiki: 91.8973
BERT base unique 4-grams relative to Wiki: 98.6718
BERT base unique 2-grams relative to self: 62.7488
BERT base unique 3-grams relative to self: 92.5906
BERT base unique 4-grams relative to self: 98.4108


In [ ]:
max_n = 4

pct_uniques_GPT = ref_unique_ngrams(openai_sents_out, wiki_data, max_n)
for i in range(2, max_n + 1):
    print("OpenAI GPT unique %d-grams relative to Wiki: %.4f" % (i, 100 * pct_uniques_GPT[i]))
    gramTable.set_cell(item =format(100 * pct_uniques_GPT[i],".4f"), row_name="GPT",col_name="WT103 unique %d-grams"%(i))

pct_uniques_GPT_self = self_unique_ngrams(openai_sents_out, max_n)
for i in range(2, max_n + 1):
    print("OpenAI GPT unique %d-grams relative to self: %.4f" % (i, 100 * pct_uniques_GPT_self[i]))
    gramTable.set_cell(item =format(100 * pct_uniques_GPT_self[i],".4f"), row_name="GPT",col_name="Self unique %d-grams"%(i))

OpenAI GPT unique 2-grams relative to Wiki: 33.6686
OpenAI GPT unique 3-grams relative to Wiki: 73.2306
OpenAI GPT unique 4-grams relative to Wiki: 91.2346
OpenAI GPT unique 2-grams relative to self: 31.5774
OpenAI GPT unique 3-grams relative to self: 67.5123
OpenAI GPT unique 4-grams relative to self: 87.6764


In [ ]:
max_n = 4

pct_uniques_tfxl = ref_unique_ngrams(transformerXL_sents_out, wiki_data, max_n)
for i in range(2, max_n + 1):
    print("TFXL %d-grams relative to Wiki: %.4f" % (i, 100 * pct_uniques_tfxl[i]))
    gramTable.set_cell(item =format(100 * pct_uniques_tfxl[i],".4f"), row_name="TFXL",col_name="WT103 unique %d-grams"%(i))

pct_uniques_self_tfxl = self_unique_ngrams(transformerXL_sents_out, max_n)
for i in range(2, max_n + 1):
    print("TFXL unique %d-grams relative to self: %.4f" % (i, 100 * pct_uniques_self_tfxl[i]))
    gramTable.set_cell(item =format(100 * pct_uniques_self_tfxl[i],".4f"), row_name="TFXL",col_name="Self unique %d-grams"%(i))

TFXL 2-grams relative to Wiki: 38.6328
TFXL 3-grams relative to Wiki: 78.1598
TFXL 4-grams relative to Wiki: 94.5532
TFXL unique 2-grams relative to self: 48.1990
TFXL unique 3-grams relative to self: 82.0207
TFXL unique 4-grams relative to self: 95.0062


In [ ]:
max_n = 4

pct_uniques_WT103 = ref_unique_ngrams(WT103_sents_out, wiki_data, max_n)
for i in range(2, max_n + 1):
    print("WT103 unique %d-grams relative to Wiki: %.4f" % (i, 100 * pct_uniques_WT103[i]))
    gramTable.set_cell(item =format(100 * pct_uniques_WT103[i],".4f"), row_name="WT103",col_name="WT103 unique %d-grams"%(i))

pct_uniques_self_WT103 = self_unique_ngrams(WT103_sents_out, max_n)
for i in range(2, max_n + 1):
    print("WT103 unique %d-grams relative to self: %.4f" % (i, 100 * pct_uniques_self_WT103[i]))
    gramTable.set_cell(item =format(100 * pct_uniques_self_WT103[i],".4f"), row_name="WT103",col_name="Self unique %d-grams"%(i))

WT103 unique 2-grams relative to Wiki: 56.3098
WT103 unique 3-grams relative to Wiki: 88.4135
WT103 unique 4-grams relative to Wiki: 97.6938
WT103 unique 2-grams relative to self: 69.1396
WT103 unique 3-grams relative to self: 93.8131
WT103 unique 4-grams relative to self: 98.8779


## EmbSim (Embedding Similarity)

EMbSimaims to quantify the similarity of two distinctwritten  texts  by  comparing  their  word  embeddings.First, word embedding is evaluated on real data using askip-gram model. For each word embedding, we com-pute its cosine distance with the other words, and thenformulate it as a matrixW, whereWij= cos(ei, ej),withei,ej, being the world embeddings of wordsiandjfrom real data.Wis calledsimilarity matrixof thereal data. The similarity matrix of the generated data isdefined analogously:W′ij= cos(e′i, e′j), wheree′i,e′jare the embeddings of wordsi′andj′from generateddata, respectively. The EmbSim is then defined as

\begin{equation}
\text{EmbSim} = \log \left(\sum_{i=1}^N \frac{\cos\left(W_i',W_i\right)}{N} \right),
\end{equation}

where $N$ is the total number of words and $W_i$ and $W_i'$ denote the $i$-th column of $W$ and $W'$ respectively.


In [ ]:
# coding=utf-8
import nltk

def text_to_code(tokens, dictionary, seq_len):
    code_str = ""
    eof_code = len(dictionary)
    for sentence in tokens:
        index = 0
        for word in sentence:
            code_str += (str(dictionary[word]) + ' ')
            index += 1
        while index < seq_len:
            code_str += (str(eof_code) + ' ')
            index += 1
        code_str += '\n'
    return code_str


def code_to_text(codes, dictionary):
    paras = ""
    eof_code = len(dictionary)
    for line in codes:
        numbers = map(int, line)
        for number in numbers:
            if number == eof_code:
                continue
            paras += (dictionary[str(number)] + ' ')
        paras += '\n'
    return paras


def get_tokenlized(file):
    tokenlized = list()
    with open(file) as raw:
        for text in raw:
            text = nltk.word_tokenize(text.lower())
            tokenlized.append(text)
    return tokenlized


def get_word_list(tokens):
    word_set = list()
    for sentence in tokens:
        for word in sentence:
            word_set.append(word)
    return list(set(word_set))


def get_dict(word_set):
    word_index_dict = dict()
    index_word_dict = dict()
    index = 0
    for word in word_set:
        word_index_dict[word] = str(index)
        index_word_dict[str(index)] = word
        index += 1
    return word_index_dict, index_word_dict

def text_precess(train_text_loc, test_text_loc=None):
    train_tokens = get_tokenlized(train_text_loc)
    if test_text_loc is None:
        test_tokens = list()
    else:
        test_tokens = get_tokenlized(test_text_loc)
    word_set = get_word_list(train_tokens + test_tokens)
    [word_index_dict, index_word_dict] = get_dict(word_set)

    if test_text_loc is None:
        sequence_len = len(max(train_tokens, key=len))
    else:
        sequence_len = max(len(max(train_tokens, key=len)), len(max(test_tokens, key=len)))
    with open('eval_data.txt', 'w') as outfile:
        outfile.write(text_to_code(test_tokens, word_index_dict, sequence_len))

    return sequence_len, len(word_index_dict) + 1

In [ ]:
import nltk
nltk.download('punkt')
sequence_len, vocab_size_BERT = text_precess(out_file_BERT, "wiki103.5k.prepared.txt")
sequence_len, vocab_size_OPENAI = text_precess(out_file_OPENAI, "wiki103.5k.prepared.txt")
sequence_len, vocab_size_TFXL = text_precess("tfxl_prepared", "wiki103.5k.prepared.txt")
sequence_len, vocab_size_WT103 = text_precess('newWiki1000.prepared.txt', "wiki103.5k.prepared.txt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import collections
import math
import random

import numpy as np
import tensorflow as tf
from scipy.spatial.distance import cosine


class DocEmbSim():
    def __init__(self, oracle_file=None, generator_file=None, num_vocabulary = None):
        super().__init__()
        self.name = 'EmbeddingSimilarity'
        self.oracle_sim = None
        self.gen_sim = None
        self.is_first = True
        self.oracle_file = oracle_file
        self.generator_file = generator_file
        self.num_vocabulary = num_vocabulary
        self.batch_size = 64
        self.embedding_size = 32
        self.data_index = 0
        self.valid_examples = None

    def get_score(self):
        if self.is_first:
            self.get_oracle_sim()
            self.is_first = False
        self.get_gen_sim()
        return self.get_dis_corr()
   
    def get_tokenlized(self,file):
        tokenlized = list()
        with open(file) as raw:
            for text in raw:
                text = nltk.word_tokenize(text.lower())
                tokenlized.append(text)
        return tokenlized

    def text_precess(self,train_text_loc):
        train_tokens = get_tokenlized(train_text_loc)
        word_set = get_word_list(train_tokens)
        [word_index_dict, index_word_dict] = get_dict(word_set)
        sequence_len = len(max(train_tokens, key=len))
        with open('eval_data.txt', 'w') as outfile:
          outfile.write(text_to_code(train_tokens, word_index_dict, sequence_len))

    def get_frequent_word(self):
        if self.valid_examples is not None:
            return self.valid_examples

        import collections
        self.text_precess(self.oracle_file)
        words = []
        with open('eval_data.txt', 'r') as file:
            for line in file:
                text = nltk.word_tokenize(line)
                text = list(map(int, text))
                words += text
        counts = collections.Counter(words)
        new_list = sorted(words, key=lambda x: -counts[x])
        word_set = list(set(new_list))
        if len(word_set) < self.num_vocabulary//10: # // divide e restituisce la parte intera -> floor division
            self.valid_examples = word_set
            return word_set
        else:
            self.valid_examples = word_set[0:self.num_vocabulary//10]
            return word_set[0:self.num_vocabulary//10]

    def read_data(self, file):
        self.text_precess(file)
        words = []
        with open('eval_data.txt', 'r') as file:
            for line in file:
                text = nltk.word_tokenize(line)
                words.append(text)
        return words

    def generate_batch(self, batch_size, num_skips, skip_window, data=None):
        assert batch_size % num_skips == 0
        assert num_skips <= 2 * skip_window
        batch = np.ndarray(shape=(batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
        span = 2 * skip_window + 1  # [ skip_window target skip_window ]
        buffer = collections.deque(maxlen=span)
        for _ in range(span):
            buffer.append(data[self.data_index])
            self.data_index = (self.data_index + 1) % len(data)
        for i in range(batch_size // num_skips):
            target = skip_window  # target label at the center of the buffer
            targets_to_avoid = [skip_window]
            for j in range(num_skips):
                while target in targets_to_avoid:
                    target = random.randint(0, span - 1)
                targets_to_avoid.append(target)
                batch[i * num_skips + j] = buffer[skip_window]
                labels[i * num_skips + j, 0] = buffer[target]
            buffer.append(data[self.data_index])
            self.data_index = (self.data_index + 1) % len(data)
        return batch, labels

    def get_wordvec(self, file):
        graph = tf.Graph()
        batch_size = self.batch_size
        embedding_size = self.embedding_size
        vocabulary_size = self.num_vocabulary
        num_sampled = 64
        if num_sampled > vocabulary_size:
            num_sampled = vocabulary_size
        num_steps = 2
        skip_window = 1  # How many words to consider left and right.
        num_skips = 2  # How many times to reuse an input to generate a label.
        if self.valid_examples is None:
            self.get_frequent_word()

        with graph.as_default():

            # Input data.
            train_dataset = tf.compat.v1.placeholder(tf.int32, shape=[batch_size])
            train_labels = tf.compat.v1.placeholder(tf.int32, shape=[batch_size, 1])
            valid_dataset = tf.compat.v1.constant(self.valid_examples, dtype=tf.int32)

            # initial Variables.
            embeddings = tf.Variable(
                tf.compat.v1.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0, seed=11))
            softmax_weights = tf.Variable(
                tf.compat.v1.truncated_normal([vocabulary_size, embedding_size],
                                    stddev=1.0 / math.sqrt(embedding_size), seed=12))
            softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))

            # Model.
            # Look up embeddings for inputs.
            embed = tf.nn.embedding_lookup(embeddings, train_dataset)
            # Compute the softmax loss, using a sample of the negative labels each time.
            loss = tf.reduce_mean(
                tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                                           labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

            optimizer = tf.compat.v1.train.AdagradOptimizer(1.0).minimize(loss)

            # Compute the similarity between minibatch examples and all embeddings.
            norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
            normalized_embeddings = embeddings / norm
            valid_embeddings = tf.nn.embedding_lookup(
                normalized_embeddings, valid_dataset)
            similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

            data = self.read_data(file)

        with tf.compat.v1.Session(graph=graph) as session:
            tf.compat.v1.global_variables_initializer().run()
            average_loss = 0
            generate_num = len(data)
            for step in range(num_steps):
                for index in range(generate_num):
                    cur_batch_data, cur_batch_labels = self.generate_batch(
                        batch_size, num_skips, skip_window, data[index])
                    feed_dict = {train_dataset: cur_batch_data, train_labels: cur_batch_labels}
                    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
                    average_loss += l
            similarity_value = similarity.eval()
            return similarity_value

    def get_oracle_sim(self):
        self.oracle_sim = self.get_wordvec(self.oracle_file)

    def get_gen_sim(self):
        self.gen_sim = self.get_wordvec(self.generator_file)

    def get_dis_corr(self):
        if len(self.oracle_sim) != len(self.gen_sim):
            raise ArithmeticError
        corr = 0
        for index in range(len(self.oracle_sim)):
            corr += (cosine(np.array(self.oracle_sim[index]), np.array(self.gen_sim[index])))
        return np.log10(corr / len(self.oracle_sim))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
objEmbSimBERTbase = DocEmbSim(oracle_file=out_file_BERT, generator_file="wiki103.5k.prepared.txt", num_vocabulary = vocab_size_BERT)
BERTEmbSim = objEmbSimBERTbase.get_score()
print("BERT base EmbSim: %.4f" % BERTEmbSim )
table.set_cell(item =format(BERTEmbSim,".4f"), row_name="BERT base",col_name="EmbSim WT103")

BERT base EmbSim: -2.1656


In [ ]:
objEmbSimOPENAI = DocEmbSim(oracle_file=out_file_OPENAI, generator_file="wiki103.5k.prepared.txt", num_vocabulary = vocab_size_OPENAI)
openaiEmbSim = objEmbSimOPENAI.get_score()
print("GPT EmbSim: %.4f" % openaiEmbSim )
table.set_cell(item =format(openaiEmbSim,".4f"), row_name="GPT",col_name="EmbSim WT103")

GPT EmbSim: -1.7255


In [ ]:
objEmbSimTFXL = DocEmbSim(oracle_file="tfxl_prepared", generator_file="wiki103.5k.prepared.txt", num_vocabulary = vocab_size_TFXL)
tfxlEmbSim = objEmbSimTFXL.get_score()
print("TFXL EmbSim: %.4f" % tfxlEmbSim )
table.set_cell(item =format(tfxlEmbSim,".4f"), row_name="TFXL",col_name="EmbSim WT103")

TFXL EmbSim: -2.2129


In [ ]:
objEmbSimWT103 = DocEmbSim(oracle_file='newWiki1000.prepared.txt', generator_file="wiki103.5k.prepared.txt", num_vocabulary = vocab_size_WT103)
WT103EmbSim = objEmbSimWT103.get_score()
print("WT103 EmbSim: %.4f" % WT103EmbSim )
table.set_cell(item =format(WT103EmbSim,".4f"), row_name="WT103",col_name="EmbSim WT103")

WT103 EmbSim: -2.4209


# Results

In [ ]:
table.print()

-------------------------------------------------------------------------------------------
|                 |Corpus-BLEU WT103|       PPL       |   Corpus-PPL    |  EmbSim WT103   |
|    BERT base    |     9.0017      |    289.1632     |    212.9936     |     -2.1656     |
|       GPT       |     11.1340     |    175.7630     |    155.2120     |     -1.7255     |
|      TFXL       |     22.0293     |    125.6344     |     66.6718     |     -2.2129     |
|      WT103      |     16.1486     |    186.1305     |     66.3768     |     -2.4209     |
-------------------------------------------------------------------------------------------


In [ ]:
gramTable.print()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|                    |     Self-BLEU      |Self unique 2-grams |Self unique 3-grams |Self unique 4-grams |WT103 unique 2-grams|WT103 unique 3-grams|WT103 unique 4-grams|
|     BERT base      |       8.7787       |      62.7488       |      92.5906       |      98.4108       |      58.5522       |      91.8973       |      98.6718       |
|        GPT         |      39.3600       |      31.5774       |      67.5123       |      87.6764       |      33.6686       |      73.2306       |      91.2346       |
|        TFXL        |      22.1123       |      48.1990       |      82.0207       |      95.0062       |      38.6328       |      78.1598       |      94.5532       |
|       WT103        |      10.1783       |      69.1396       |      93.8131       |      98.8779       |      56.3098       |      88.4135       |  

In [2]:
tfxlscores =list()
gptscores = list()
bertbasescores = list()
with open("progettoML/fluencyscores.txt","r") as f:
  for line in f.readlines():
    ll=line.split()
    bertbasescores.append(ll[1])
    gptscores.append(ll[2])
    tfxlscores.append(ll[3])
bertbasescores = bertbasescores[1:]
gptscores = gptscores[1:]
tfxlscores = tfxlscores[1:]

In [3]:
import plotly.graph_objects as go

import numpy as np
x0 = np.random.randn(500)
x1 = np.random.randn(500) + 1

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=bertbasescores,
    histnorm='percent',
    name='BERT base',
    xbins=dict(
        start=0.97,
        end=4.27,
        size=0.33
    ),
    marker_color='#F9AF08',
    opacity=0.75
))

fig.add_trace(go.Histogram(
    x=gptscores,
    histnorm='percent',
    name='GPT', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=0.97,
        end=4.27,
        size=0.33
    ),
    marker_color='#08F91B',
    opacity=0.75
))

fig.add_trace(go.Histogram(
    x=tfxlscores,
    histnorm='percent',
    name='TFXL',
    xbins=dict(
        start=0.97,
        end=4.27,
        size=0.33
    ),
    marker_color='#08C4F9',
    opacity=0.75
))

fig.update_layout(
    #title_text='Sampled Results', # title of plot
    yaxis_title_text='Number of sentences', # xaxis label
    xaxis_title_text='Score', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.update_xaxes(ticktext=["","1.0<br> (Not fluent)", "1.5", "2.0 <br> (Somewhat fluent)", "2.5","3.0 <br> (Fluent)","3.5","4.0 <br> (Very fluent)",""],
                 tickvals=[0.5,1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0,4.5])

fig.show()

# VANILLA MLE

First select 5000 rows by the 1000 of the train of Vanilla MLE as a basiline.

In [ ]:
import re
from numpy.random import default_rng
import numpy as np

# [lines_to_pick] indica il numero di righe da restituire
# Se supera il numero massimo di righe che sono state preprocessate le ritorna tutte
# Se lines_to_pick == -1 le prende tutte in ogni caso
def preprocess_file_MLE(filename: str, lines_to_pick=-1):#, min_line_length=7, max_line_length=100):
    with open(filename, 'r') as f:
        content = f.readlines()
    print("Initial data:", len(content))
    without_newline = ("".join(content)).split("\n")
    print("Splitted on new line")
    no_empty_string = [line.strip() for line in without_newline if len(
        line.split(" ")) > 1 and len(line) != 1]
    print("Removed empty strings")
    
    #filtered_unk = [item for item in no_empty_string if "<unk>" not in  item.split(" ")]

    #print("Filtered for length between {} and {}".format(
    #    min_line_length, max_line_length))

    count = len(no_empty_string)
    if lines_to_pick == -1 or lines_to_pick >= count:
        print("Returning all lines")
        return no_empty_string
    else:
        print("Returning {} lines".format(lines_to_pick))
        indexes = default_rng().choice(count - 1, size=lines_to_pick, replace=False)
        return [no_empty_string[index] for index in indexes]

In [ ]:
#lines = preprocess_file_MLE('progettoML/Texygen-master/data/wiki.train.txt', lines_to_pick=1000)
#with open('wikiSelfPointMLE.txt','w+')as f:
#    for line in lines:
#       f.write(line)
#       f.write("\n")

Initial data: 10000
Splitted on new line
Removed empty strings
Returning 1000 lines


In [ ]:
out_file_WT103MLE = 'progettoML/Texygen-master/save/wikiSelfPointMLE.txt'
WT103MLE_sents_out = [sent.replace("<unk>","[UNK]").strip().split() for sent in open(out_file_WT103MLE).readlines()]
for sent in WT103MLE_sents_out[0:5]:
  print(" ".join(sent))

with open("wikiSelfPointMLE_prepared.txt",'w') as f:
  for sent in WT103MLE_sents_out:
    f.write(" ".join(sent))
    f.write("\n")

Linked metallocenes can also be formed by introducing several [UNK] substituents onto a single cyclopentadienyl ligand ..
" Like the former , Adrien Begrand of PopMatters wrote enthusiastically for both Cavalera Conspiracy and Inflikted ..
It has been suggested that the " spirit hands " in the séances of Home were made of gloves stuffed with a substance.
Evidence during a board of inquiry later showed that it was a friendly fire incident involving U.S. Air Force and U.S. Navy aircraft mistaking the ships for enemy targets.
The cave @-@ dwelling predators capture crustaceans under 1 mm ( 0 @.@ 039 in ) long by entangling them with fine threads , digest them by enveloping them with further threads over the course of a few days , and then return to their normal shape ; there is no evidence that they use venom ..


Compute perplexity of WT103MLE:

In [ ]:
PPL_WT103MLE = np.average([(perplexity_model.score(" ".join(sent))['positional_scores'].mean().neg().exp()).item() for sent in WT103MLE_sents_out])
print("WT103MLE perplexity:",format(PPL_WT103MLE,".4f"))
tableMLE.set_cell(item=format(PPL_WT103MLE,".4f"), row_name="WT103MLE",col_name="PPL")

WT103MLE perplexity: 129.5462


In [ ]:
PPL_WT103MLE_corpus = [perplexity_model.score(" ".join(sent))['positional_scores'].mean().item() for sent in WT103MLE_sents_out]
PPL_WT103MLE_corpus = np.exp(-np.mean(PPL_WT103MLE_corpus))
print("WT103MLE perplexity-corpus:",format(PPL_WT103MLE_corpus,".4f"))
tableMLE.set_cell(item=format(PPL_WT103MLE_corpus,".4f"), row_name="WT103MLE",col_name="Corpus-PPL")
WT103MLE_sents_out = [sent.lower().replace("<unk>","[UNK]").strip().split() for sent in open(out_file_WT103MLE).readlines()]

WT103MLE perplexity-corpus: 71.1355


Import generated sentence by Vanilla MLE.

In [ ]:
#lines = preprocess_file_MLE('progettoML/Texygen-master/save/test_fileMLE.txt', lines_to_pick=1000)
#with open('generatedMLE1000.txt','w+')as f:
#    for line in lines:
#       f.write(line)
#       f.write("\n")

Initial data: 9984
Splitted on new line
Removed empty strings
Returning 1000 lines


In [ ]:
out_file_MLE = 'progettoML/Texygen-master/save/generatedMLE1000.txt'
MLE_sents_out = [sent.replace("< unk >","[UNK]").replace("@ . @","@.@").replace("@ , @","@,@").replace("@ - @","@-@").replace("``","\"").strip().split() for sent in open(out_file_MLE).readlines()]
for sent in MLE_sents_out[0:5]:
  print(" ".join(sent))

with open("generatedMLE1000_prepared.txt",'w') as f:
  for sent in MLE_sents_out:
    f.write(" ".join(sent))
    f.write("\n")

" one of hardcore ray bc , is the bill was unsuccessfully for 18 @.@ 2 km ) market capitalization is affected by ken marino , and tape boy shotgun , indicating that new show that frances is also noted that the film was would be seen .
chinese welding also in a series is featured in the traditional tree @-@ mouthed mamba , is something .
the agreement five was that he was without impossible , the fate of the world and deliver at a friendly district show and jma upgrading the southwest floppy victory and and aft ) .
oxford felt that it is strengthened , along by the supplier of the hand west of his daughter ( debut , and dancers recorded " , was , with land to be dangerous ..
he was too forced middleton much as a clerk of 25 standing against the right master @-@ ten in the abd al @-@ eup and angang @-@ shirts cross southern swimming in finland and of order to the nation more through one moving on too level into a key men 's hollywood .


In [ ]:
PPL_MLE = np.average([(perplexity_model.score(" ".join(sent))['positional_scores'].mean().neg().exp()).item() for sent in MLE_sents_out])
print("MLE perplexity:",format(PPL_MLE,".4f"))
tableMLE.set_cell(item=format(PPL_MLE,".4f"), row_name="MLE",col_name="PPL")

MLE perplexity: 3285.5876


In [ ]:
PPL_MLE_corpus = [perplexity_model.score(" ".join(sent))['positional_scores'].mean().item() for sent in MLE_sents_out]
PPL_MLE_corpus = np.exp(-np.mean(PPL_MLE_corpus))
print("MLE perplexity-corpus:",format(PPL_MLE_corpus,".4f"))
tableMLE.set_cell(item=format(PPL_MLE_corpus,".4f"), row_name="MLE",col_name="Corpus-PPL")

MLE perplexity-corpus: 1649.8054


In [ ]:
bleuMLE = 100 * corpus_bleu(MLE_sents_out, wiki_data)
print("MLE bleu corpus:",format(bleuMLE,".4f"))
tableMLE.set_cell(item=format(bleuMLE,".4f"), row_name="MLE",col_name="Corpus-BLEU WT103")

MLE bleu corpus: 11.5018


In [ ]:
bleuWT103MLE = 100 * corpus_bleu(WT103MLE_sents_out, wiki_data)
print("WT103MLE bleu corpus:",format(bleuWT103MLE,".4f"))
tableMLE.set_cell(item=format(bleuWT103MLE,".4f"), row_name="WT103MLE",col_name="Corpus-BLEU WT103")

WT103MLE bleu corpus: 13.3151


In [ ]:
selfbleuMLE= 100 * self_bleu(MLE_sents_out)
print("MLE self-bleu:",format(selfbleuMLE,".4f"))
gramTableMLE.set_cell(item=format(selfbleuMLE,".4f"), row_name="MLE",col_name="Self-BLEU")

MLE self-bleu: 10.2983


In [ ]:
selfbleuWT103MLE = 100 * self_bleu(WT103MLE_sents_out)
print("WT103MLE self-bleu:",format(selfbleuWT103MLE,".4f"))
gramTableMLE.set_cell(item=format(selfbleuWT103MLE,".4f"), row_name="WT103MLE",col_name="Self-BLEU")

WT103MLE self-bleu: 10.0048


In [ ]:
max_n = 4

pct_uniques_MLE = ref_unique_ngrams(MLE_sents_out, wiki_data, max_n)
for i in range(2, max_n + 1):
    print("MLE %d-grams relative to Wiki: %.4f" % (i, 100 * pct_uniques_MLE[i]))
    gramTableMLE.set_cell(item =format(100 * pct_uniques_MLE[i],".4f"), row_name="MLE",col_name="WT103 unique %d-grams"%(i))

pct_uniques_self_MLE = self_unique_ngrams(MLE_sents_out, max_n)
for i in range(2, max_n + 1):
    print("MLE unique %d-grams relative to self: %.4f" % (i, 100 * pct_uniques_self_MLE[i]))
    gramTableMLE.set_cell(item =format(100 * pct_uniques_self_MLE[i],".4f"), row_name="MLE",col_name="Self unique %d-grams"%(i))

MLE 2-grams relative to Wiki: 57.5716
MLE 3-grams relative to Wiki: 91.2157
MLE 4-grams relative to Wiki: 99.0350
MLE unique 2-grams relative to self: 65.4716
MLE unique 3-grams relative to self: 94.0357
MLE unique 4-grams relative to self: 99.3602


In [ ]:
max_n = 4

pct_uniques_WT103MLE = ref_unique_ngrams(WT103MLE_sents_out, wiki_data, max_n)
for i in range(2, max_n + 1):
    print("WT103MLE unique %d-grams relative to Wiki: %.4f" % (i, 100 * pct_uniques_WT103MLE[i]))
    gramTableMLE.set_cell(item =format(100 * pct_uniques_WT103MLE[i],".4f"), row_name="WT103MLE",col_name="WT103 unique %d-grams"%(i))

pct_uniques_self_WT103MLE = self_unique_ngrams(WT103MLE_sents_out, max_n)
for i in range(2, max_n + 1):
    print("WT103MLE unique %d-grams relative to self: %.4f" % (i, 100 * pct_uniques_self_WT103MLE[i]))
    gramTableMLE.set_cell(item =format(100 * pct_uniques_self_WT103MLE[i],".4f"), row_name="WT103MLE",col_name="Self unique %d-grams"%(i))

WT103MLE unique 2-grams relative to Wiki: 56.9304
WT103MLE unique 3-grams relative to Wiki: 88.4208
WT103MLE unique 4-grams relative to Wiki: 97.8288
WT103MLE unique 2-grams relative to self: 69.3997
WT103MLE unique 3-grams relative to self: 94.1645
WT103MLE unique 4-grams relative to self: 98.9331


In [ ]:
sequence_len, vocab_size_MLE = text_precess("generatedMLE1000_prepared.txt", "wiki103.5k.prepared.txt")
sequence_len, vocab_size_WT103MLE = text_precess("wikiSelfPointMLE_prepared.txt", "wiki103.5k.prepared.txt")

In [ ]:
objEmbSimMLE = DocEmbSim(oracle_file="generatedMLE1000_prepared.txt", generator_file="wiki103.5k.prepared.txt", num_vocabulary = vocab_size_MLE)
MLEEmbSim = objEmbSimMLE.get_score()
print("MLE EmbSim: %.4f" % MLEEmbSim )
tableMLE.set_cell(item =format(MLEEmbSim,".4f"), row_name="MLE",col_name="EmbSim")

MLE EmbSim: -2.3916


In [ ]:
objEmbSimWT103MLE = DocEmbSim(oracle_file="wikiSelfPointMLE_prepared.txt", generator_file="wiki103.5k.prepared.txt", num_vocabulary = vocab_size_WT103MLE)
WT103MLEEmbSim = objEmbSimWT103MLE.get_score()
print("WT103 EmbSim: %.4f" % WT103MLEEmbSim )
tableMLE.set_cell(item =format(WT103MLEEmbSim,".4f"), row_name="WT103MLE",col_name="EmbSim")

WT103 EmbSim: -2.3850


In [ ]:
tableMLE.print()

-------------------------------------------------------------------------------------------
|                 |Corpus-BLEU WT103|       PPL       |   Corpus-PPL    |     EmbSim      |
|       MLE       |     11.5018     |    3285.5876    |    1649.8054    |     -2.3916     |
|    WT103MLE     |     13.3151     |    129.5462     |     71.1355     |     -2.3850     |
-------------------------------------------------------------------------------------------


In [ ]:
gramTableMLE.print()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|                    |     Self-BLEU      |Self unique 2-grams |Self unique 3-grams |Self unique 4-grams |WT103 unique 2-grams|WT103 unique 3-grams|WT103 unique 4-grams|
|        MLE         |      10.2983       |      65.4716       |      94.0357       |      99.3602       |      57.5716       |      91.2157       |      99.0350       |
|      WT103MLE      |      10.0048       |      69.3997       |      94.1645       |      98.9331       |      56.9304       |      88.4208       |      97.8288       |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
